## *Jeu de données IRVE*
-------------------
# Analyse des relations entre champs (intégrité des données)
------------------------------
## Contexte
- clarification du rôle des modèles de données dans les jeux de données (cf mise à jour récente des [guides data.gouv](https://guides.etalab.gouv.fr/qualite/documenter-les-donnees/))
- intégration d'une propriété "relationship" dans les schémas de données ([issue TableSchema](https://github.com/frictionlessdata/specs/issues/803) en cours de validation)
- création d'outils de contrôle des relations entre champs des jeux de données tabulaires (cf usage ci-dessous)

## Objectifs
- valider sur un cas réel l'utilisation d'un modèle de données en complément d'un schéma de données
- identifier les apports que pourraient avoir les contrôles de validation des relations entre champs

## Résultats
- les principaux résultats sont regroupés sur un [tableau de bord](https://nbviewer.org/github/loco-philippe/Environmental-Sensing/blob/main/python/Validation/irve/Analyse/IRVE_indicateurs.ipynb) 
- la formalisation d'un modèle de données facilite la compréhension des données et des relations entre champs
- l'outil de contrôle permet d'améliorer significativement la qualité des données par l'identification d'incohérences de relations
- l'identification des incohérences permet de trouver des stratégies de réduction des écarts (dans l'exemple ci-dessous, on passe 36% d'écart à 3,8 %)  
- l'analyse des données permet de (re)construire le modèle de données qui minimise les incohérences
- les incohérences détectées sur le jeu de données IRVE restent faibles (inférieures à 5 % des point de charge documentés - voir chapitre 4)

## Suite à donner
- Mettre à jour, valider et publier le modèle de données IRVE
- Définir les contrôles supplémentaires à intégrer pour toutes nouvelles données ainsi que pour le jeu complet
- Mettre en oeuvre les outils de contrôle

## Evolutions possibles 
- Ajouter dans les guides d'Etalab un guide pour les modèles de données 
- Intégrer dans les schémas de données la propriété "relationship" en cours de validation,
- Définir un indicateur qui mesure l'écart (existant / attendu) des relations entre champs

## Sommaire
*(liens actifs sur jupyter Notebook ou Nbviewer)*
- [1 - modèle de données](#1---modèle-de-données)
- [2 - Initialisation](#2---Initialisation)
- [3 - Séparation des pdc itinerance et hors itinerance](#3---Séparation-des-pdc-itinerance-et-hors-itinerance)
- [4 - Bilan initial intégrité](#4---Bilan-initial-intégrité)
- [5 - Séparation doublons pdc - date de maj](#5---Séparation-doublons-pdc---date-de-maj)
- [6 - Séparation doublons station - date de maj](#6---Séparation-doublons-station---date-de-maj)
- [7 - Synthèse](#7---Synthèse)
- [8 - Exemples d'erreurs résiduelles](#8---Exemples-d\'erreurs-résiduelles)

Ce Notebook peut être consulté sur [nbviewer](http://nbviewer.org/github/loco-philippe/Environmental-Sensing/tree/main/python/Validation/irve/Analyse)

données utilisées : https://www.data.gouv.fr/fr/datasets/fichier-consolide-des-bornes-de-recharge-pour-vehicules-electriques/    
fichier : "*consolidation-etalab-schema-irve-statique-v-2.2.0-2024xxxx.csv*"

-----------------------
# 1 - modèle de données

Le modèle de données proposé ci-dessous est construit sur la base du schéma de données mis à disposition et du contenu du jeu de données.    
Il est à consolider en fonction de l'expertise des concepteurs et réutilisateurs (voir [guide méthodologique](https://github.com/loco-philippe/Environmental-Sensing/blob/main/property_relationship/FR_methodology.ipynb)).

*Notation:*
- *M : Mandatory - documentation obligatoire*
- *PK : Primary Key - identifiant unique de l'entité*
- *Root : champ fictif associé à une ligne du tableau*

In [1]:
from base64 import b64encode
from IPython.display import Image, display
with open('IRVE_modele.txt', 'r', encoding="utf-8") as f:
    modele = f.read()
display(Image(url="https://mermaid.ink/img/" + b64encode(modele.encode("ascii")).decode("ascii")))

------
# 2 - Initialisation
## initialisation logicielle

In [64]:
from datetime import datetime
import json
from tab_dataset import Cdataset
import pandas as pd
import ntv_pandas as npd
import pathlib

def analyse_integrite(data, schema, affiche=True, indic=True):
    '''analyse les relations du DataFrame 'data' définies dans le schéma 'schema'.
    Le nombre de ligne en erreur par relation (dict) est retourné et optionnellement affiché (paramètre 'affiche=True') . 
    Les lignes en erreur sont optionnellement ajoutées (paramètre 'indic=True') à 'data' sous forme de champs booléens par relation.
    '''
    dic_errors = Cdataset(data).check_relationship(schema)
    dic_count = {name: len(errors) for name, errors in dic_errors.items()}
    if affiche:
        for name, total in dic_count.items():
            print('{:<50} {:>5}'.format(name, total))
    if indic:
        data['ok'] = True
        for name, errors in dic_errors.items():
            data[name] = True
            data.loc[errors, name] = False
            data['ok'] = data['ok'] & data[name] 
    return dic_count

def add_nbre_pdc(data, affiche=True):
    '''ajoute un champ avec le nombre de pdc calculés et le compare au champ 'nbre_pdc'
    Le nombre d'écarts est affiché optionnellement (paramètre 'affiche=True').
    '''
    data['nb_pdc_calc'] = data.groupby('id_station_itinerance')['index'].transform('count')
    data['nbre_pdc_ok'] = data['nb_pdc_calc'] == data['nbre_pdc']    
    data['calc-nbre'] = data['nb_pdc_calc'] - data['nbre_pdc']
    if affiche:
        ecart_nbre = len(data) - data['nbre_pdc_ok'].sum()
        print('{:<20} {:>5}'.format('nbre_pdc_ko', ecart_nbre))
        print(round(ecart_nbre / len(data) * 100), ' %')

## initialisation des données
- lecture du fichier issu de l'api

In [65]:
dates = ['2023-03-03', '2023-04-17', '2023-05-02', '2023-05-24', '2023-06-17', '2023-07-04', '2023-07-23', '2023-08-08',
         '2023-09-06', '2023-09-19', '2023-10-04', '2023-10-21', '2023-10-31', '2023-11-23', '2023-12-14', '2023-12-29',
         '2024-01-15', '2024-02-07', '2024-02-28', '2024-03-27', '2024-04-24', '2024-05-19', '2024-06-05']
date= dates[-1]
log = {'date_irve': date, 
    'file': 'consolidation-etalab-schema-irve-statique-v-2.3.1-'+date[:4]+date[5:7]+date[8:]+'.csv', # après 2024-02-06
    #'file': 'consolidation-etalab-schema-irve-statique-v-2.2.0-'+date[:4]+date[5:7]+date[8:]+'.csv',
    'chemin': str(pathlib.Path(npd.__file__).parent.parent.parent/"Environmental-Sensing"/"python"/"Validation"/"irve"/"Analyse")
    # 'chemin': 'https://raw.githubusercontent.com/loco-philippe/Environmental-Sensing/gen-ntv/python/Validation/irve/Analyse/'}
    # log = {'date_irve': '2022-06-06', 'file': 'consolidation-etalab-schema-irve-v-2.0.2-20220606-propre2.csv',
    #'chemin': 'D:\\philippe\\python ESstandard\\Environmental-Sensing\\python\\Validation\\irve\\Analyse\\'
    #'chemin': 'C:\\Users\\phili\\github\\Environmental-Sensing\\python\\Validation\\irve\\Analyse\\'
      }
irve = pd.read_csv(log['chemin'] + '/' + log['file'], sep=',', low_memory=False)
log['len_irve'] = len(irve)
print('nombre de lignes : ', log['len_irve']) 

nombre de lignes :  77733


## schéma de données
Le schéma de données restreint à la propriété 'relationship' et construit à partir du modèle de données est le suivants :

In [66]:
# complément à inclure dans le schéma de données
schema = {
    'relationships': [
         # relation unicité des pdl
         {"fields": ["id_pdc_itinerance", "index"],                    "link" : "coupled" },   
         # relations inter entités
         {"fields": ["id_station_itinerance", "contact_operateur"],    "link" : "derived" },
         {"fields": ["id_station_itinerance", "nom_enseigne"],         "link" : "derived" },
         {"fields": ["id_station_itinerance", "coordonneesXY"],        "link" : "derived" },
         {"fields": ["id_pdc_itinerance", "id_station_itinerance"],    "link" : "derived" },
         # relations intra entité - station
         {"fields": ["id_station_itinerance", "nom_station"],          "link" : "derived" },
         {"fields": ["id_station_itinerance", "implantation_station"], "link" : "derived" },
         #{"fields": ["id_station_itinerance", "date_maj"],             "link" : "derived" },
         {"fields": ["id_station_itinerance", "nbre_pdc"],             "link" : "derived" },
         {"fields": ["id_station_itinerance", "condition_acces"],      "link" : "derived" },
         {"fields": ["id_station_itinerance", "horaires"],             "link" : "derived" },
         {"fields": ["id_station_itinerance", "station_deux_roues"],   "link" : "derived" },
         # relations intra entité - localisation
         {"fields": ["coordonneesXY", "adresse_station"],              "link" : "derived" }
    ]
}
# liste des champs liés à un controle (relations) et obligatoires (mandatory)
relations = ['index', 'contact_operateur', 'nom_enseigne', 'coordonneesXY', 'adresse_station', 'id_station_itinerance', 
             'nom_station', 'implantation_station', 'nbre_pdc', 'condition_acces', 'horaires', 'station_deux_roues', 
             'id_pdc_itinerance', 'date_maj', 'last_modified']
mandatory = ['contact_operateur', 'nom_enseigne', 'coordonneesXY', 'adresse_station', 'id_station_itinerance', 'nom_station',
             'implantation_station', 'nbre_pdc', 'condition_acces', 'horaires', 'station_deux_roues', 'id_pdc_itinerance', 
             'puissance_nominale', 'prise_type_ef', 'prise_type_2', 'prise_type_combo_ccs', 'prise_type_chademo', 
             'prise_type_autre', 'paiement_acte', 'paiement_autre', 'reservation',  'accessibilite_pmr', 'restriction_gabarit', 
             'date_maj', 'last_modified']
controles = ['index - id_pdc_itinerance', 'contact_operateur - id_station_itinerance', 'nom_enseigne - id_station_itinerance',
             'coordonneesXY - id_station_itinerance', 'id_station_itinerance - id_pdc_itinerance',
             'nom_station - id_station_itinerance', 'implantation_station - id_station_itinerance',
             'nbre_pdc - id_station_itinerance', 'condition_acces - id_station_itinerance', 'horaires - id_station_itinerance',
             'station_deux_roues - id_station_itinerance', 'adresse_station - coordonneesXY']

-----------------------
## 3 - Séparation des pdc itinerance et hors itinerance
- de l'ordre de 3 % des points de charge sont hors itinerance

In [67]:
# séparation des données
data = irve
data[['id_station_itinerance','id_pdc_itinerance']] = data[['id_station_itinerance','id_pdc_itinerance']].astype('string')
#data[['last_modified','date_maj']] = data[['last_modified','date_maj']].astype('datetime64')
data['non_concerne'] = data['id_station_itinerance'].str.contains('oncern') | data['id_pdc_itinerance'].str.contains('oncern')

non_concerne = data[data['non_concerne']].reset_index()['index']
itinerance = data[~data['non_concerne']].reset_index()
itinerance_init = itinerance.loc[:, relations]
log['pdc_hors_itinerance'] = len(non_concerne)
log['pdc_en_itinerance'] = len(itinerance)
print('nombre de pdc hors itinerance : ', log['pdc_hors_itinerance'], 'soit : ', len(non_concerne)/(len(non_concerne)+len(itinerance)))
print('nombre de pdc en itinerance   : ', log['pdc_en_itinerance'])

nombre de pdc hors itinerance :  2909 soit :  0.037422973511893276
nombre de pdc en itinerance   :  74824


In [68]:
# arborescence des champs et nombre de valeurs différentes
print(itinerance_init.npd.analysis().tree())

-1: root-derived (74824)
   0 : index (0 - 74824)
   1 : contact_operateur (74667 - 157)
   2 : nom_enseigne (71662 - 3162)
   3 : coordonneesXY (56179 - 18645)
   4 : adresse_station (55671 - 19153)
   5 : id_station_itineranc (40622 - 34202)
   6 : nom_station (55479 - 19345)
   7 : implantation_station (74819 - 5)
   8 : nbre_pdc (74763 - 61)
   9 : condition_acces (74822 - 2)
   10: horaires (74312 - 512)
   11: station_deux_roues (74818 - 6)
   12: id_pdc_itinerance (2338 - 72486)
   13: date_maj (74124 - 700)
   14: last_modified (74305 - 519)


-----------------------
## 4 - Bilan initial intégrité
- 22 % des lignes présentent un défaut d'intégrité

In [69]:
# séparation données bonnes (itinerance_ok_1) et données résiduelles (itinerance_1)
res = analyse_integrite(itinerance_init, schema)
itinerance_ok_1 = itinerance_init.loc[itinerance_init.ok, relations].reset_index(drop=True)
itinerance_1 = itinerance_init.loc[~itinerance_init.ok, relations].reset_index(drop=True)
itinerance_init = itinerance_init.loc[:, relations]
log['init_ok'] = len(itinerance_ok_1)
log['init_ko'] = len(itinerance_1)
print("\nnombre d'enregistrements sans erreurs : ", log['init_ok'])
print("nombre d'enregistrements avec au moins une erreur : ", log['init_ko'])
print("taux d'erreur : ", round(log['init_ko'] / log['pdc_en_itinerance'] * 100), ' %')

index - id_pdc_itinerance                           4674
contact_operateur - id_station_itinerance             53
nom_enseigne - id_station_itinerance                 666
coordonneesXY - id_station_itinerance               1353
id_station_itinerance - id_pdc_itinerance           4034
nom_station - id_station_itinerance                 1358
implantation_station - id_station_itinerance         238
nbre_pdc - id_station_itinerance                    1488
condition_acces - id_station_itinerance               65
horaires - id_station_itinerance                      85
station_deux_roues - id_station_itinerance           789
adresse_station - coordonneesXY                     5862

nombre d'enregistrements sans erreurs :  63948
nombre d'enregistrements avec au moins une erreur :  10876
taux d'erreur :  15  %


-----------------------
## 5 - Séparation doublons pdc - date de maj
- 35% des pdc en erreur sont liées aux doublons de pdc
- la suppression des doublons permet de réduire de 50% le nombre de lignes erronnées

In [70]:
# séparation doublons pdc (doublons_pdc) et données résiduelles (itinerance_2)
itinerance_1['doublons_pdc'] = itinerance_1.sort_values(by=['date_maj', 'last_modified']).duplicated('id_pdc_itinerance', keep='last')

doublons_pdc = itinerance_1[itinerance_1['doublons_pdc']].loc[:, relations].reset_index(drop=True)['index']
#doublons_pdc = itinerance_1[itinerance_1['doublons_pdc']].loc[:, relations].reset_index(drop=True)
itinerance_2 = itinerance_1[~itinerance_1['doublons_pdc']].loc[:, relations].reset_index(drop=True)
itinerance_1 = itinerance_1.loc[:, relations]
log['doublons_pdc'] = len(doublons_pdc)
log['sans_doublons_pdc'] = len(itinerance_2)
print('nombre de doublons pdc : ', log['doublons_pdc'], ' soit ', round(log['doublons_pdc']/log['init_ko'] * 100), ' %')
print('nombre de pdc sans doublon   : ', log['sans_doublons_pdc'], ' soit ', round(log['sans_doublons_pdc']/log['init_ko'] * 100), ' %')

nombre de doublons pdc :  2338  soit  21  %
nombre de pdc sans doublon   :  8538  soit  79  %


In [71]:
# séparation données bonnes (itinerance_ok_3) et données résiduelles (itinerance_3)
res = analyse_integrite(itinerance_2, schema)
itinerance_ok_3 = itinerance_2.loc[itinerance_2.ok, relations].reset_index(drop=True)
itinerance_3 = itinerance_2.loc[~itinerance_2.ok, relations].reset_index(drop=True)
itinerance_2 = itinerance_2.loc[:, relations]
log['etape3_ok'] = len(itinerance_ok_3)
log['etape3_ko'] = len(itinerance_3)
print("\nnombre d'enregistrements sans erreurs : ", log['etape3_ok'])
print("nombre d'enregistrements avec au moins une erreur : ", log['etape3_ko'])

index - id_pdc_itinerance                              0
contact_operateur - id_station_itinerance             24
nom_enseigne - id_station_itinerance                  33
coordonneesXY - id_station_itinerance                711
id_station_itinerance - id_pdc_itinerance              0
nom_station - id_station_itinerance                 1322
implantation_station - id_station_itinerance         227
nbre_pdc - id_station_itinerance                     985
condition_acces - id_station_itinerance               38
horaires - id_station_itinerance                      70
station_deux_roues - id_station_itinerance           789
adresse_station - coordonneesXY                     4775

nombre d'enregistrements sans erreurs :  2281
nombre d'enregistrements avec au moins une erreur :  6257


-----------------------
## 6 - Séparation doublons station - date de maj
- la suppression des anciens pdc permet de réduire de 25% supplémentaire le nombre de lignes erronnées
- les dernières erreurs correspondent à des stations associées à 43 opérateurs et sont liées à des causes multiples

In [72]:
# séparation doublons stations (doublons_stat_maj) et données résiduelles (itinerance_4)
itinerance_3['stat_maj'] = itinerance_3.id_station_itinerance + itinerance_3.date_maj
stat_maj_unique = itinerance_3.sort_values(by='stat_maj').drop_duplicates('id_station_itinerance', keep='last')
itinerance_3['last_stat_maj'] = itinerance_3['stat_maj'].isin(stat_maj_unique['stat_maj'])

doublons_stat_maj = itinerance_3[~itinerance_3['last_stat_maj']].loc[:, relations].reset_index(drop=True)['index']
itinerance_4 = itinerance_3[itinerance_3['last_stat_maj']].loc[:, relations].reset_index(drop=True)
itinerance_3 = itinerance_3.loc[:, relations]
log['doublons_station'] = len(doublons_stat_maj)
log['sans_doublons_station'] = len(itinerance_4)
print('nombre de doublons stations : ', log['doublons_station'], ' soit ', 
      round(log['doublons_station']/log['init_ko'] * 100), ' %')
print('nombre de pdc sans doublon   : ', log['sans_doublons_station'])

nombre de doublons stations :  1346  soit  12  %
nombre de pdc sans doublon   :  4911


In [73]:
# séparation données bonnes (itinerance_ok_5) et données résiduelles (itinerance_5 / itinerance_5_full)
res = analyse_integrite(itinerance_4, schema)
itinerance_ok_5 = itinerance_4.loc[itinerance_4.ok, relations].reset_index(drop=True)
itinerance_5_full = itinerance_4.loc[~itinerance_4.ok].reset_index(drop=True)
itinerance_5 = itinerance_5_full.loc[:, relations]
#itinerance_4 = itinerance_4.loc[:, relations]
log['etape5_ok'] = len(itinerance_ok_5)
log['etape5_ko'] = len(itinerance_5)
print("\nnombre d'enregistrements sans erreurs : ", log['etape5_ok'])
print("nombre d'enregistrements avec au moins une erreur : ", log['etape5_ko'], ' soit ', 
      round(log['etape5_ko']/log['etape3_ko'] * 100), ' %')

index - id_pdc_itinerance                              0
contact_operateur - id_station_itinerance              2
nom_enseigne - id_station_itinerance                   5
coordonneesXY - id_station_itinerance                269
id_station_itinerance - id_pdc_itinerance              0
nom_station - id_station_itinerance                  321
implantation_station - id_station_itinerance         199
nbre_pdc - id_station_itinerance                     374
condition_acces - id_station_itinerance               18
horaires - id_station_itinerance                      67
station_deux_roues - id_station_itinerance           448
adresse_station - coordonneesXY                     2345

nombre d'enregistrements sans erreurs :  1337
nombre d'enregistrements avec au moins une erreur :  3574  soit  57  %


In [74]:
# structure des données présentant une erreur et nombre de valeurs
print(itinerance_5.npd.analysis().tree())

-1: root-derived (3574)
   0 : index (0 - 3574)
   1 : contact_operateur (3524 - 50)
   2 : nom_enseigne (3381 - 193)
   3 : coordonneesXY (2877 - 697)
   4 : adresse_station (2416 - 1158)
   5 : id_station_itineranc (1789 - 1785)
      13: date_maj (1690 - 95)
   6 : nom_station (2512 - 1062)
   7 : implantation_station (3569 - 5)
   8 : nbre_pdc (3539 - 35)
   9 : condition_acces (3572 - 2)
   10: horaires (3492 - 82)
   11: station_deux_roues (3568 - 6)
   12: id_pdc_itinerance (0 - 3574)
   14: last_modified (3483 - 91)


In [75]:
# nb maxi d'erreurs
itinerance_4['somme'] = 0
for name in res.keys():
    itinerance_4['somme'] += 1 - itinerance_4[name]
erreurs = max(itinerance_4['somme'])
maxi = itinerance_4[itinerance_4.somme >= erreurs]
print("nombre d'enregistrements avec ", erreurs, " erreurs : ", len(maxi))

nombre d'enregistrements avec  4  erreurs :  11


In [171]:
# principaux opérateurs contributeur de défauts
operateurs = list(itinerance_5.drop_duplicates('contact_operateur')['contact_operateur'])
erreurs_op = [len(itinerance_5.loc[itinerance_5.contact_operateur == operateur]) for operateur in operateurs]
err_op_5, oper_5 = tuple(zip(*(sorted(zip(erreurs_op, operateurs), reverse=True))[:5]))

print(len(err_op_5), ' opérateurs représentent : ', round(sum(err_op_5) / sum(erreurs_op) * 100), ' % des écarts\n')
for err, op in zip(err_op_5, oper_5):
    print(err, op)

5  opérateurs représentent :  71  % des écarts

804 support@alizecharge.fr
584 supervision-ev.france@totalenergies.com
428 info@ionity.eu
408 assistance-technique@ouestcharge-paysdelaloire-moncompte.fr
322 sav@izivia.com


-----------------------
## 7 - Synthèse


### fichiers
Génération des fichiers intégrant les défauts d'intégrité :
- fichier csv des lignes résiduelles à traiter (IRVE_itinerance_residuel)
- fichier csv des données itinerance avec indicateur des données à corriger ou à ignorer (IRVE_itinerance_complet)
- fichier csv des données itinerance valides (IRVE_itinerance_valide)
- fichier csv des doublons (IRVE_itinerance_doublons)

In [76]:
# consolidation des données
itinerance['doublons_stat_maj'] = itinerance['index'].isin(doublons_stat_maj)
itinerance['doublons_pdc'] = itinerance['index'].isin(doublons_pdc)
itinerance['lignes_a_corriger'] = itinerance['index'].isin(itinerance_5['index'])
itinerance['doublons_a_supprimer'] = itinerance['doublons_stat_maj'] | itinerance['doublons_pdc']
itinerance['lignes_ko'] = itinerance['doublons_a_supprimer'] | itinerance['lignes_a_corriger']
print('total des lignes à corriger : ', itinerance['lignes_a_corriger'].sum())
itinerance_doublons = itinerance[itinerance['doublons_a_supprimer']].reset_index(drop=True)
print('total des doublons à supprimer : ', len(itinerance_doublons))
itinerance_ok = itinerance[~itinerance['lignes_ko']].reset_index(drop=True)
print('nombre de pdc avec controles ok : ', len(itinerance_ok))

total des lignes à corriger :  3574
total des doublons à supprimer :  3684
nombre de pdc avec controles ok :  67566


In [77]:
# génération des fichiers
extension = log['date_irve'] +'.csv'
itinerance_5_full.to_csv('IRVE_itinerance_residuel' + extension)
itinerance.to_csv('IRVE_itinerance_complet' + extension)
itinerance_ok.to_csv('IRVE_itinerance_valide' + extension)
itinerance_doublons.to_csv('IRVE_itinerance_doublons' + extension)
log['IRVE_itinerance_residuel' + extension] = len(itinerance_5_full)
log['IRVE_itinerance_complet' + extension] = len(itinerance)
log['IRVE_itinerance_valide' + extension] = len(itinerance_ok)
log['IRVE_itinerance_valide_stat' + extension] = len(itinerance_ok.drop_duplicates('id_station_itinerance', keep='last'))
log['IRVE_itinerance_doublons' + extension] = len(itinerance_doublons)

In [173]:
for op in oper_5:
    print('IRVE_itinerance_residuel' + '_' + op + '_' + extension)
    itinerance_5_full.loc[itinerance_5_full.contact_operateur == op].to_csv('IRVE_itinerance_residuel' + '_' + op + '_' + extension)

IRVE_itinerance_residuel_support@alizecharge.fr_2024-06-05.csv
IRVE_itinerance_residuel_supervision-ev.france@totalenergies.com_2024-06-05.csv
IRVE_itinerance_residuel_info@ionity.eu_2024-06-05.csv
IRVE_itinerance_residuel_assistance-technique@ouestcharge-paysdelaloire-moncompte.fr_2024-06-05.csv
IRVE_itinerance_residuel_sav@izivia.com_2024-06-05.csv


### vérification de l'intégrité

In [78]:
# vérification de l'absence d'erreurs
res = analyse_integrite(itinerance_ok.loc[:, relations], schema, affiche=False)
log['bilan_erreurs'] = sum(res.values())
log['date'] = datetime.now().isoformat()
print('bilan intégrité :')
print('    erreurs : ', log['bilan_erreurs'])

bilan intégrité :
    erreurs :  0


In [79]:
# structure des données bonnes
#print(Cdataset(itinerance_ok.loc[:, mandatory]).tree())
print(itinerance_ok.loc[:, mandatory].npd.analysis().tree())

-1: root-derived (67566)
   4 : id_station_itineranc (35554 - 32012)
      1 : nom_enseigne (28982 - 3030)
      2 : coordonneesXY (14255 - 17757)
         0 : contact_operateur (17611 - 146)
         3 : adresse_station (348 - 17409)
      5 : nom_station (14163 - 17849)
      6 : implantation_station (32007 - 5)
      7 : nbre_pdc (31951 - 61)
      8 : condition_acces (32010 - 2)
      9 : horaires (31569 - 443)
      10: station_deux_roues (32006 - 6)
      19: paiement_autre (32003 - 9)
      22: restriction_gabarit (31906 - 106)
      24: last_modified (31546 - 466)
   11: id_pdc_itinerance (0 - 67566)
   12: puissance_nominale (67465 - 101)
   13: prise_type_ef (67558 - 8)
   14: prise_type_2 (67558 - 8)
   15: prise_type_combo_ccs (67559 - 7)
   16: prise_type_chademo (67559 - 7)
   17: prise_type_autre (67559 - 7)
   18: paiement_acte (67558 - 8)
   20: reservation (67560 - 6)
   21: accessibilite_pmr (67562 - 4)
   23: date_maj (66897 - 669)


### Indicateurs

In [80]:
# génération des indicateurs liés aux règles
regles = ['Pdc non unique', 'Station multi-operateurs', 'Station multi-enseignes', 'Station multi-localisations', 
          'Pdc multi-stations', 'station avec plusieurs noms', 'station multi-implantations', 
          'nombre de pdc par station incoherent', 'station multi-acces', 'station multi-horaires', 
          'acces deux-roues incoherent', 'localisation multi-adresses']
principal = [16, 17, 18, 19, 20]
secondaire = [21, 22, 23, 24, 25, 26, 27]
irve = itinerance_5_full # residuel
total = len(irve)
indic = {}

irve['principal'] = True
for ind in principal:
    irve['principal'] &= irve.iloc[:,ind]
    indic[regles[ind-16]] = int(total - irve.iloc[:,ind].sum())
irve['secondaire'] = True
for ind in secondaire:
    irve['secondaire'] &= irve.iloc[:,ind]
    indic[regles[ind-16]] = int(total - irve.iloc[:,ind].sum())
irve['secondaire'] |= (~irve['principal'] & ~irve['secondaire'])
irve['verif'] = irve['principal'] & irve['secondaire']
indic['principal pdc'] = int(total - irve['principal'].sum())
indic['secondaire pdc'] = int(total - irve['secondaire'].sum())

irve_p = irve[~irve['principal']].drop_duplicates('id_station_itinerance').reset_index(drop=True)
irve_s = irve[~irve['secondaire']].drop_duplicates('id_station_itinerance').reset_index(drop=True)
indic['principal stat'] = len(irve_p)
indic['secondaire stat'] = len(irve_s)

In [ ]:
# ajout des principaux opérateurs contributeur de défauts
indic['operateurs'] = list(oper_5)
indic['erreurs_operateurs'] = list(err_op_5)

In [82]:
# ajout de l'indicateur d'écart entre nombre de pdc calculés et saisis
add_nbre_pdc(irve, affiche=True)
indic['nb_pdc_calc-nbre_pdc'] = int(irve['calc-nbre'].sum())

nbre_pdc_ko           1763
49  %


In [83]:
# indicateur d'écart entre nombre de pdc calculés et saisis sur les données sans doublon
itinerance_sans_doublons = itinerance[~itinerance['doublons_a_supprimer']].reset_index(drop=True)
add_nbre_pdc(itinerance_sans_doublons, affiche=True)

nbre_pdc_ko          26860
38  %


In [84]:
len(itinerance_4)

4911

In [85]:
# stockage des indicateurs
log |= indic
log_file = log['chemin'] + '/' + 'logfile.txt'
#with open('logfile.txt', 'a', encoding="utf-8") as f:
with open(log_file, 'a', encoding="utf-8") as f:
    f.write(json.dumps(log) + '\n')

In [162]:
itinerance_5_full.columns[16:28]

Index(['index - id_pdc_itinerance',
       'contact_operateur - id_station_itinerance',
       'nom_enseigne - id_station_itinerance',
       'coordonneesXY - id_station_itinerance',
       'id_station_itinerance - id_pdc_itinerance',
       'nom_station - id_station_itinerance',
       'implantation_station - id_station_itinerance',
       'nbre_pdc - id_station_itinerance',
       'condition_acces - id_station_itinerance',
       'horaires - id_station_itinerance',
       'station_deux_roues - id_station_itinerance',
       'adresse_station - coordonneesXY'],
      dtype='object')

-----------------------
## 8 - Exemples d'erreurs résiduelles 

### Erreurs multiples 
- exemple avec le nombre maximal d'erreurs (3) (109 pdc)
- exemple station 'FR55CP92140' avec 29 pdc ! : 5 coordonnées XY, 5 nom_station, 2 nbre_pdc)

In [86]:
maxi.loc[:, relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
66,740,contact@e55c.com,ELECTRIC 55 CHARGING,"[5.976272, 45.526596]",BP - AIRE DE L'ABIS - SAINT-JEOIRE-PRIEURE - 7...,FR55CPBP73190AB1S,BP - AIRE DE L'ABIS - SAINT-JEOIRE-PRIEURE - 7...,Voirie,3,Accès réservé,24/7,FALSE,FR55CEFR7319043AB1S0,2022-11-03,2024-05-16T11:29:02.272000+00:00
67,741,contact@e55c.com,ELECTRIC 55 CHARGING,"[5.976272, 45.526596]",BP - AIRE DE L'ABIS - SAINT-JEOIRE-PRIEURE - 7...,FR55CPBP73190AB1S,BP - AIRE DE L'ABIS - SAINT-JEOIRE-PRIEURE - 7...,Voirie,3,Accès réservé,24/7,FALSE,FR55CEFR7319043AB1S1,2022-11-03,2024-05-16T11:29:02.272000+00:00
68,742,contact@e55c.com,ELECTRIC 55 CHARGING,"[5.976272, 45.526596]",BP - AIRE DE L'ABIS - SAINT-JEOIRE-PRIEURE - 7...,FR55CPBP73190AB1S,BP - AIRE DE L'ABIS - SAINT-JEOIRE-PRIEURE - 7...,Voirie,3,Accès réservé,24/7,FALSE,FR55CEFR7319043AB1S2,2022-11-03,2024-05-16T11:29:02.272000+00:00
69,743,contact@e55c.com,ELECTRIC 55 CHARGING,"[5.976272, 45.526596]",BP - AIRE DE L'ABIS SAINT-JEOIRE-PRIEURE,FR55CPBP73190AB1S,BP - AIRE DE L'ABIS SAINT-JEOIRE-PRIEURE - 73190,Station dédiée à la recharge rapide,8,Accès réservé,24/7,FALSE,FR55CEFR7319043AB1S3,2022-11-03,2024-05-16T11:29:02.272000+00:00
70,744,contact@e55c.com,ELECTRIC 55 CHARGING,"[5.976272, 45.526596]",BP - AIRE DE L'ABIS SAINT-JEOIRE-PRIEURE,FR55CPBP73190AB1S,BP - AIRE DE L'ABIS SAINT-JEOIRE-PRIEURE - 73190,Station dédiée à la recharge rapide,8,Accès réservé,24/7,FALSE,FR55CEFR7319043AB1S4,2022-11-03,2024-05-16T11:29:02.272000+00:00
71,745,contact@e55c.com,ELECTRIC 55 CHARGING,"[5.976272, 45.526596]",BP - AIRE DE L'ABIS SAINT-JEOIRE-PRIEURE,FR55CPBP73190AB1S,BP - AIRE DE L'ABIS SAINT-JEOIRE-PRIEURE - 73190,Station dédiée à la recharge rapide,8,Accès réservé,24/7,FALSE,FR55CEFR7319043AB1S5,2022-11-03,2024-05-16T11:29:02.272000+00:00
72,746,contact@e55c.com,ELECTRIC 55 CHARGING,"[5.976272, 45.526596]",BP - AIRE DE L'ABIS SAINT-JEOIRE-PRIEURE,FR55CPBP73190AB1S,BP - AIRE DE L'ABIS SAINT-JEOIRE-PRIEURE - 73190,Station dédiée à la recharge rapide,8,Accès réservé,24/7,FALSE,FR55CEFR7319043AB1S6,2022-11-03,2024-05-16T11:29:02.272000+00:00
73,747,contact@e55c.com,ELECTRIC 55 CHARGING,"[5.976272, 45.526596]",BP - AIRE DE L'ABIS SAINT-JEOIRE-PRIEURE,FR55CPBP73190AB1S,BP - AIRE DE L'ABIS SAINT-JEOIRE-PRIEURE - 73190,Station dédiée à la recharge rapide,8,Accès réservé,24/7,FALSE,FR55CEFR7319043AB1S7,2022-11-03,2024-05-16T11:29:02.272000+00:00
74,748,contact@e55c.com,ELECTRIC 55 CHARGING,"[5.976272, 45.526596]",BP - AIRE DE L'ABIS SAINT-JEOIRE-PRIEURE,FR55CPBP73190AB1S,BP - AIRE DE L'ABIS SAINT-JEOIRE-PRIEURE - 73190,Station dédiée à la recharge rapide,8,Accès réservé,24/7,FALSE,FR55CEFR7319043AB1S8,2022-11-03,2024-05-16T11:29:02.272000+00:00
75,749,contact@e55c.com,ELECTRIC 55 CHARGING,"[5.976272, 45.526596]",BP - AIRE DE L'ABIS SAINT-JEOIRE-PRIEURE,FR55CPBP73190AB1S,BP - AIRE DE L'ABIS SAINT-JEOIRE-PRIEURE - 73190,Station dédiée à la recharge rapide,8,Accès réservé,24/7,FALSE,FR55CEFR7319043AB1S9,2022-11-03,2024-05-16T11:29:02.272000+00:00


In [87]:
itinerance_4.loc[itinerance_4.id_station_itinerance == 'FR55CP92140', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified


### Repartition des defauts par operateur
Répartition pour les 5 opérateurs présentant le plus de défauts

In [88]:
print('PdC avec une incohérence : ')
for operateur in oper:
    print('\n', sum(itinerance_5.contact_operateur == operateur))
    print('Répartition des incohérences ', operateur, ' :')
    for cont in controles:
        #print(cont, sum((itinerance_4['contact_operateur'] == 'supervision-ev.france@totalenergies.com') & ~(itinerance_4[cont])))
        print('{:<50} {:>5}'.format(cont, sum((itinerance_4['contact_operateur'] == operateur) & ~(itinerance_4[cont]))))

PdC avec une incohérence : 

 804
Répartition des incohérences  support@alizecharge.fr  :
index - id_pdc_itinerance                              0
contact_operateur - id_station_itinerance              0
nom_enseigne - id_station_itinerance                   0
coordonneesXY - id_station_itinerance                118
id_station_itinerance - id_pdc_itinerance              0
nom_station - id_station_itinerance                  193
implantation_station - id_station_itinerance          71
nbre_pdc - id_station_itinerance                     107
condition_acces - id_station_itinerance                0
horaires - id_station_itinerance                       0
station_deux_roues - id_station_itinerance             0
adresse_station - coordonneesXY                      602

 584
Répartition des incohérences  supervision-ev.france@totalenergies.com  :
index - id_pdc_itinerance                              0
contact_operateur - id_station_itinerance              0
nom_enseigne - id_station_itinera

### Defauts operateur Ouestcharge-Paysdelaloire
Les défauts identifiés concernent des stations utilisant un même identifiant. Par exemple, la station 'FRADPP91479018' contient 82 points de recharge avec plusieurs adresses et un nombre de points de rechage variable. Cet exemple concerne des stations implantées sur plusieurs étages d'un parking (on devrait donc a minima identifier une station par étage).

In [125]:
sum(itinerance_sans_doublons.contact_operateur == 'assistance-technique@ouestcharge-paysdelaloire-moncompte.fr')

726

In [90]:
itinerance_4.loc[itinerance_4['contact_operateur'] == 'assistance-technique@ouestcharge-paysdelaloire-moncompte.fr', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
3103,56163,assistance-technique@ouestcharge-paysdelaloire...,SYDELA,"[-1.397512, 47.074964]",36 Avenue de la Vendée 44140 AIGREFEUILLE-SUR-...,FRS44P44002B,OuestCharge - Diva Sp - Aigrefeuille-Sur-Maine...,Voirie,1,Accès libre,24/7,false,FRS44E44002A1,2023-09-07,2024-01-19T07:46:12.032000+00:00
3104,56164,assistance-technique@ouestcharge-paysdelaloire...,SYDELA,"[-1.397512, 47.074964]",36 Avenue de la Vendée 44140 AIGREFEUILLE-SUR-...,FRS44P44002B,OuestCharge - Diva Sp - Aigrefeuille-Sur-Maine...,Voirie,1,Accès libre,24/7,false,FRS44E44002A2,2023-09-07,2024-01-19T07:46:12.032000+00:00
3105,56165,assistance-technique@ouestcharge-paysdelaloire...,SYDELA,"[-1.410933, 47.081675]",Rue de la Filée 44140 AIGREFEUILLE-SUR-MAINE (44),FRS44P44002A,OuestCharge - Pulse 50 - Aigrefeuille-Sur-Main...,Voirie,1,Accès libre,24/7,false,FRS44E44002B1,2023-09-07,2024-01-19T07:46:12.032000+00:00
3110,56170,assistance-technique@ouestcharge-paysdelaloire...,SYDELA,"[-1.177845, 47.370906]",67 Boulevard du Docteur Moutel 44150 ANCENIS (44),FRS44P44003B,OuestCharge - Diva Sp - Ancenis - Docteur Moutel,Voirie,1,Accès libre,24/7,false,FRS44E44003A1,2023-09-07,2024-01-19T07:46:12.032000+00:00
3111,56171,assistance-technique@ouestcharge-paysdelaloire...,SYDELA,"[-1.177845, 47.370906]",67 Boulevard du Docteur Moutel 44150 ANCENIS (44),FRS44P44003B,OuestCharge - Diva Sp - Ancenis - Docteur Moutel,Voirie,1,Accès libre,24/7,false,FRS44E44003A2,2023-09-07,2024-01-19T07:46:12.032000+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4020,62493,assistance-technique@ouestcharge-paysdelaloire...,SYDEV,"[-1.022143, 46.877453]",Avenue De L'Europe - Parking Origne Hall 85500...,FRS85P85109A,OuestCharge - 50 Compact - Les Herbiers - Europe,Voirie,2,Accès libre,24/7,false,FRS85E85109AB2CCS,2024-04-22,2024-04-22T09:38:03.113000+00:00
4021,62494,assistance-technique@ouestcharge-paysdelaloire...,SYDEV,"[-1.022143, 46.877453]",Avenue De L'Europe - Parking Origne Hall 85500...,FRS85P85109A,OuestCharge - 50 Compact - Les Herbiers - Europe,Voirie,2,Accès libre,24/7,false,FRS85E85109AB2CHADEMO,2024-04-22,2024-04-22T09:38:03.113000+00:00
4022,62495,assistance-technique@ouestcharge-paysdelaloire...,SYDEV,"[-1.022143, 46.877453]",Avenue De L'Europe - Parking Origne Hall 85500...,FRS85P85109A,OuestCharge - 50 Compact - Les Herbiers - Europe,Voirie,2,Accès libre,24/7,false,FRS85E85109AB2T2,2024-04-22,2024-04-22T09:38:03.113000+00:00
4025,62629,assistance-technique@ouestcharge-paysdelaloire...,SYDEV,"[-1.13728, 46.811699]",Les 4 Chemins de l'Oie 85140 SAINTE-FLORENCE (85),FRS85P85212A,OuestCharge - 50 Compact - Sainte-Florence - Oie,Voirie,2,Accès libre,24/7,false,FRS85E852120013,2024-04-22,2024-04-22T09:38:03.113000+00:00


In [91]:
itinerance_4.loc[itinerance_4.coordonneesXY == '[-1.397512, 47.074964]', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
3101,56161,support@alizecharge.fr,Ouest Charge,"[-1.397512, 47.074964]","36 Avenue de la Vendée, 44140 AIGREFEUILLE-SUR...",FRS44E44002002,AIGREFEUILLE-SUR-MAINE - Avenue De La Vendée,Voirie,2,Accès libre,Mo-Su 00:00-23:57,False,FRS44E440020021,2024-05-22,2024-05-22T15:33:49.735000+00:00
3102,56162,support@alizecharge.fr,Ouest Charge,"[-1.397512, 47.074964]","36 Avenue de la Vendée, 44140 AIGREFEUILLE-SUR...",FRS44E44002002,AIGREFEUILLE-SUR-MAINE - Avenue De La Vendée,Voirie,2,Accès libre,Mo-Su 00:00-23:57,False,FRS44E440020022,2024-05-22,2024-05-22T15:33:49.735000+00:00
3103,56163,assistance-technique@ouestcharge-paysdelaloire...,SYDELA,"[-1.397512, 47.074964]",36 Avenue de la Vendée 44140 AIGREFEUILLE-SUR-...,FRS44P44002B,OuestCharge - Diva Sp - Aigrefeuille-Sur-Maine...,Voirie,1,Accès libre,24/7,false,FRS44E44002A1,2023-09-07,2024-01-19T07:46:12.032000+00:00
3104,56164,assistance-technique@ouestcharge-paysdelaloire...,SYDELA,"[-1.397512, 47.074964]",36 Avenue de la Vendée 44140 AIGREFEUILLE-SUR-...,FRS44P44002B,OuestCharge - Diva Sp - Aigrefeuille-Sur-Maine...,Voirie,1,Accès libre,24/7,false,FRS44E44002A2,2023-09-07,2024-01-19T07:46:12.032000+00:00


### Defauts operateur ALIZECHARGE
Les défauts identifiés concernent des stations utilisant un même identifiant. Par exemple, la station 'FRADPP91479018' contient 82 points de recharge avec plusieurs adresses et un nombre de points de rechage variable. Cet exemple concerne des stations implantées sur plusieurs étages d'un parking (on devrait donc a minima identifier une station par étage).

In [148]:
itinerance_4.loc[itinerance_4['contact_operateur'] == 'support@alizecharge.fr', relations][50:200]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
143,1953,support@alizecharge.fr,ADP Groupe,"[2.362554, 48.728787]","2 Rue des Transporteurs , 94390 Paray-Vieille-...",FRADPE914790184,ORY - Parking P3 SILO - Niveau 4,Parking public,16,Accès libre,Mo-Su 00:00-23:57,False,FRADPE9147901847,2024-05-22,2024-05-22T15:33:49.735000+00:00
144,1954,support@alizecharge.fr,ADP Groupe,"[2.362554, 48.728787]","2 Rue des Transporteurs , 94390 Paray-Vieille-...",FRADPE914790184,ORY - Parking P3 SILO - Niveau 4,Parking public,16,Accès libre,Mo-Su 00:00-23:57,False,FRADPE9147901848,2024-05-22,2024-05-22T15:33:49.735000+00:00
145,1955,support@alizecharge.fr,ADP Groupe,"[2.362554, 48.728787]","2 Rue des Transporteurs, 94390 Paray-Vieille-P...",FRADPE914790184,ORY - Parking P3 SILO - Niveau 5,Parking public,14,Accès libre,Mo-Su 00:00-23:57,False,FRADPE9147901849,2024-05-22,2024-05-22T15:33:49.735000+00:00
146,1956,support@alizecharge.fr,ADP Groupe,"[2.362554, 48.728787]","2 Rue des Transporteurs , 94390 Paray-Vieille-...",FRADPE91479018,ORY - Parking P3 SILO - Niveau 2,Parking public,16,Accès libre,Mo-Su 00:00-23:57,False,FRADPE914790185,2024-05-22,2024-05-22T15:33:49.735000+00:00
147,1957,support@alizecharge.fr,ADP Groupe,"[2.362554, 48.728787]","2 Rue des Transporteurs, 94390 Paray-Vieille-P...",FRADPE914790185,ORY - Parking P3 SILO - Niveau 5,Parking public,14,Accès libre,Mo-Su 00:00-23:57,False,FRADPE9147901850,2024-05-22,2024-05-22T15:33:49.735000+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2462,33071,support@alizecharge.fr,Grenoble-Alpes Métropole,"[5.736255, 45.15697]","Rue Henri Barbusse, 38100 GRENOBLE",FRM38E38185021,GRENOBLE - Alpexpo - Rue Henri Barbusse (CITIZ),Voirie,1,Accès libre,Mo-Su 00:00-23:57,False,FRM38E381850211,2024-05-22,2024-05-22T15:33:49.735000+00:00
2463,33072,support@alizecharge.fr,Grenoble-Alpes Métropole,"[5.736255, 45.15697]","Rue Henri Barbusse, 38100 GRENOBLE",FRM38E38185021,GRENOBLE - Alpexpo - Rue Henri Barbusse,Voirie,1,Accès libre,Mo-Su 00:00-23:57,False,FRM38E381850212,2024-05-22,2024-05-22T15:33:49.735000+00:00
2464,33096,support@alizecharge.fr,Grenoble-Alpes Métropole,"[5.6692959, 45.2427324]","15 Route de Lyon, 38120 SAINT-EGREVE",FRM38E38382002,SAINT-EGREVE - Parking Karben,Voirie,3,Accès libre,Mo-Su 00:00-23:57,False,FRM38E383820021,2024-05-22,2024-05-22T15:33:49.735000+00:00
2465,33097,support@alizecharge.fr,Grenoble-Alpes Métropole,"[5.6692959, 45.2427324]","15 Route de Lyon, 38120 SAINT-EGREVE",FRM38E38382002,SAINT-EGREVE - Parking Karben,Voirie,3,Accès libre,Mo-Su 00:00-23:57,False,FRM38E383820022,2024-05-22,2024-05-22T15:33:49.735000+00:00


In [182]:
# example [2.362554, 48.728787]
adp = itinerance_4.loc[itinerance_4.coordonneesXY == '[2.362554, 48.728787]']
adresses = list(adp.drop_duplicates('adresse_station')['adresse_station'])
print(adresses)

print(adp.loc[~adp['nom_station - id_station_itinerance'], ['nom_station', 'id_station_itinerance']])

print(adp.loc[~adp['implantation_station - id_station_itinerance'], ['implantation_station', 'id_station_itinerance']])

print(adp.loc[~adp['nbre_pdc - id_station_itinerance'], ['nbre_pdc', 'id_station_itinerance']])

itinerance_4.loc[itinerance_4.coordonneesXY == '[2.362554, 48.728787]', relations]

['2 Rue des Transporteurs , 94390 Paray-Vieille-Poste', '2 Rue des Transporteurs, 94390  Paray-Vieille-Poste', '2 Rue des Transporteurs, 94390 Paray-Vieille-Poste']
                          nom_station id_station_itinerance
103  ORY - Parking P3 SILO - Niveau 2       FRADPE914790181
104  ORY - Parking P3 SILO - Niveau 2       FRADPE914790181
105  ORY - Parking P3 SILO - Niveau 2       FRADPE914790181
106  ORY - Parking P3 SILO - Niveau 2       FRADPE914790181
107  ORY - Parking P3 SILO - Niveau 2       FRADPE914790181
108  ORY - Parking P3 SILO - Niveau 2       FRADPE914790181
109  ORY - Parking P3 SILO - Niveau 2       FRADPE914790181
110  ORY - Parking P3 SILO - Niveau 3       FRADPE914790181
111  ORY - Parking P3 SILO - Niveau 3       FRADPE914790181
112  ORY - Parking P3 SILO - Niveau 3       FRADPE914790181
125  ORY - Parking P3 SILO - Niveau 3       FRADPE914790183
126  ORY - Parking P3 SILO - Niveau 3       FRADPE914790183
127  ORY - Parking P3 SILO - Niveau 3       FRADPE91479

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
102,1912,support@alizecharge.fr,ADP Groupe,"[2.362554, 48.728787]","2 Rue des Transporteurs , 94390 Paray-Vieille-...",FRADPE91479018,ORY - Parking P3 SILO - Niveau 2,Parking public,16,Accès libre,Mo-Su 00:00-23:57,False,FRADPE914790181,2024-05-22,2024-05-22T15:33:49.735000+00:00
103,1913,support@alizecharge.fr,ADP Groupe,"[2.362554, 48.728787]","2 Rue des Transporteurs , 94390 Paray-Vieille-...",FRADPE914790181,ORY - Parking P3 SILO - Niveau 2,Parking public,16,Accès libre,Mo-Su 00:00-23:57,False,FRADPE9147901810,2024-05-22,2024-05-22T15:33:49.735000+00:00
104,1914,support@alizecharge.fr,ADP Groupe,"[2.362554, 48.728787]","2 Rue des Transporteurs , 94390 Paray-Vieille-...",FRADPE914790181,ORY - Parking P3 SILO - Niveau 2,Parking public,16,Accès libre,Mo-Su 00:00-23:57,False,FRADPE9147901811,2024-05-22,2024-05-22T15:33:49.735000+00:00
105,1915,support@alizecharge.fr,ADP Groupe,"[2.362554, 48.728787]","2 Rue des Transporteurs , 94390 Paray-Vieille-...",FRADPE914790181,ORY - Parking P3 SILO - Niveau 2,Parking public,16,Accès libre,Mo-Su 00:00-23:57,False,FRADPE9147901812,2024-05-22,2024-05-22T15:33:49.735000+00:00
106,1916,support@alizecharge.fr,ADP Groupe,"[2.362554, 48.728787]","2 Rue des Transporteurs , 94390 Paray-Vieille-...",FRADPE914790181,ORY - Parking P3 SILO - Niveau 2,Parking public,16,Accès libre,Mo-Su 00:00-23:57,False,FRADPE9147901813,2024-05-22,2024-05-22T15:33:49.735000+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
177,1987,support@alizecharge.fr,ADP Groupe,"[2.362554, 48.728787]","2 Rue des Transporteurs , 94390 Paray-Vieille-...",FRADPE91479018,ORY - Parking P3 SILO - Niveau 2,Parking public,16,Accès libre,Mo-Su 00:00-23:57,False,FRADPE914790188,2024-05-22,2024-05-22T15:33:49.735000+00:00
178,1988,support@alizecharge.fr,ADP Groupe,"[2.362554, 48.728787]","2 Rue des Transporteurs, 94390 Paray-Vieille-P...",FRADPE914790188,ORY - Parking P3 SILO - Niveau 6,Parking public,18,Accès libre,Mo-Su 00:00-23:57,False,FRADPE9147901880,2024-05-22,2024-05-22T15:33:49.735000+00:00
179,1989,support@alizecharge.fr,ADP Groupe,"[2.362554, 48.728787]","2 Rue des Transporteurs, 94390 Paray-Vieille-P...",FRADPE914790188,ORY - Parking P3 SILO - Niveau 6,Parking public,18,Accès libre,Mo-Su 00:00-23:57,False,FRADPE9147901881,2024-05-22,2024-05-22T15:33:49.735000+00:00
180,1990,support@alizecharge.fr,ADP Groupe,"[2.362554, 48.728787]","2 Rue des Transporteurs, 94390 Paray-Vieille-P...",FRADPE914790188,ORY - Parking P3 SILO - Niveau 6,Parking public,18,Accès libre,Mo-Su 00:00-23:57,False,FRADPE9147901882,2024-05-22,2024-05-22T15:33:49.735000+00:00


In [137]:
itinerance_4.loc[itinerance_4['contact_operateur'] == 'support@alizecharge.fr', relations].loc[~itinerance_4['adresse_station - coordonneesXY']][500:600]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
3768,56897,support@alizecharge.fr,Ouest Charge,"[-1.621305, 47.329273]","68 Rue de la Mairie, 44119 TREILLIERES",FRS44E44209003,TREILLIERES - Rue De La Mairie,Voirie,2,Accès libre,Mo-Su 00:00-23:57,False,FRS44E442090031,2024-05-22,2024-05-22T15:33:49.735000+00:00
3769,56898,support@alizecharge.fr,Ouest Charge,"[-1.621305, 47.329273]","68 Rue de la Mairie, 44119 TREILLIERES",FRS44E44209003,TREILLIERES - Rue De La Mairie,Voirie,2,Accès libre,Mo-Su 00:00-23:57,False,FRS44E442090032,2024-05-22,2024-05-22T15:33:49.735000+00:00
3773,56904,support@alizecharge.fr,Ouest Charge,"[-2.200174, 47.299795]","32 Rue Léo Lagrange, 44570 TRIGNAC",FRS44E44210001,TRIGNAC - Rue Léo Lagrange,Voirie,2,Accès libre,Mo-Su 00:00-23:57,False,FRS44E442100011,2024-05-22,2024-05-22T15:33:49.735000+00:00
3774,56905,support@alizecharge.fr,Ouest Charge,"[-2.200174, 47.299795]","32 Rue Léo Lagrange, 44570 TRIGNAC",FRS44E44210001,TRIGNAC - Rue Léo Lagrange,Voirie,2,Accès libre,Mo-Su 00:00-23:57,False,FRS44E442100012,2024-05-22,2024-05-22T15:33:49.735000+00:00
3775,56906,support@alizecharge.fr,Ouest Charge,"[-2.187091, 47.317501]","17 Place de La Mairie, 44570 TRIGNAC",FRS44E44210002,TRIGNAC - Place De La Mairie,Voirie,2,Accès libre,Mo-Su 00:00-23:57,False,FRS44E442100021,2024-05-22,2024-05-22T15:33:49.735000+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4014,62434,support@alizecharge.fr,SYDEV (FR*S85),"[-0.746495, 46.646595]",33 Avenue du Géneral de Gaulle - Parking Super...,FRS85E85059002,LA CHATAIGNERAIE - Avenue du Géneral de Gaulle...,Voirie,2,Accès libre,Mo-Su 00:00-23:57,False,FRS85E850590022,2024-05-22,2024-05-22T15:33:49.735000+00:00
4018,62491,support@alizecharge.fr,SYDEV (FR*S85),"[-1.022143, 46.877453]","1 Avenue De L'Europe - Parking Origne Hall, 85...",FRS85E85109004,LES HERBIERS - Avenue De L'Europe - Parking Or...,Voirie,2,Accès libre,Mo-Su 00:00-23:57,False,FRS85E851090041,2024-05-22,2024-05-22T15:33:49.735000+00:00
4019,62492,support@alizecharge.fr,SYDEV (FR*S85),"[-1.022143, 46.877453]","1 Avenue De L'Europe - Parking Origne Hall, 85...",FRS85E85109004,LES HERBIERS - Avenue De L'Europe - Parking Or...,Voirie,2,Accès libre,Mo-Su 00:00-23:57,False,FRS85E851090042,2024-05-22,2024-05-22T15:33:49.735000+00:00
4023,62625,support@alizecharge.fr,SYDEV (FR*S85),"[-1.13728, 46.811699]","Les 4 Chemins De L'Oie, 85140 SAINTE-FLORENCE",FRS85E85212001,SAINTE-FLORENCE - Les 4 Chemins De L'Oie,Voirie,2,Accès libre,Mo-Su 00:00-23:57,False,FRS85E852120011,2024-05-22,2024-05-22T15:33:49.735000+00:00


In [166]:
itinerance_4.loc[itinerance_4.id_station_itinerance == 'FRADPE914790181', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
103,1913,support@alizecharge.fr,ADP Groupe,"[2.362554, 48.728787]","2 Rue des Transporteurs , 94390 Paray-Vieille-...",FRADPE914790181,ORY - Parking P3 SILO - Niveau 2,Parking public,16,Accès libre,Mo-Su 00:00-23:57,False,FRADPE9147901810,2024-05-22,2024-05-22T15:33:49.735000+00:00
104,1914,support@alizecharge.fr,ADP Groupe,"[2.362554, 48.728787]","2 Rue des Transporteurs , 94390 Paray-Vieille-...",FRADPE914790181,ORY - Parking P3 SILO - Niveau 2,Parking public,16,Accès libre,Mo-Su 00:00-23:57,False,FRADPE9147901811,2024-05-22,2024-05-22T15:33:49.735000+00:00
105,1915,support@alizecharge.fr,ADP Groupe,"[2.362554, 48.728787]","2 Rue des Transporteurs , 94390 Paray-Vieille-...",FRADPE914790181,ORY - Parking P3 SILO - Niveau 2,Parking public,16,Accès libre,Mo-Su 00:00-23:57,False,FRADPE9147901812,2024-05-22,2024-05-22T15:33:49.735000+00:00
106,1916,support@alizecharge.fr,ADP Groupe,"[2.362554, 48.728787]","2 Rue des Transporteurs , 94390 Paray-Vieille-...",FRADPE914790181,ORY - Parking P3 SILO - Niveau 2,Parking public,16,Accès libre,Mo-Su 00:00-23:57,False,FRADPE9147901813,2024-05-22,2024-05-22T15:33:49.735000+00:00
107,1917,support@alizecharge.fr,ADP Groupe,"[2.362554, 48.728787]","2 Rue des Transporteurs , 94390 Paray-Vieille-...",FRADPE914790181,ORY - Parking P3 SILO - Niveau 2,Parking public,16,Accès libre,Mo-Su 00:00-23:57,False,FRADPE9147901814,2024-05-22,2024-05-22T15:33:49.735000+00:00
108,1918,support@alizecharge.fr,ADP Groupe,"[2.362554, 48.728787]","2 Rue des Transporteurs , 94390 Paray-Vieille-...",FRADPE914790181,ORY - Parking P3 SILO - Niveau 2,Voirie,16,Accès libre,Mo-Su 00:00-23:57,False,FRADPE9147901815,2024-05-22,2024-05-22T15:33:49.735000+00:00
109,1919,support@alizecharge.fr,ADP Groupe,"[2.362554, 48.728787]","2 Rue des Transporteurs , 94390 Paray-Vieille-...",FRADPE914790181,ORY - Parking P3 SILO - Niveau 2,Voirie,16,Accès libre,Mo-Su 00:00-23:57,False,FRADPE9147901816,2024-05-22,2024-05-22T15:33:49.735000+00:00
110,1920,support@alizecharge.fr,ADP Groupe,"[2.362554, 48.728787]","2 Rue des Transporteurs, 94390 Paray-Vieille-...",FRADPE914790181,ORY - Parking P3 SILO - Niveau 3,Parking public,16,Accès libre,Mo-Su 00:00-23:57,False,FRADPE9147901817,2024-05-22,2024-05-22T15:33:49.735000+00:00
111,1921,support@alizecharge.fr,ADP Groupe,"[2.362554, 48.728787]","2 Rue des Transporteurs, 94390 Paray-Vieille-...",FRADPE914790181,ORY - Parking P3 SILO - Niveau 3,Parking public,16,Accès libre,Mo-Su 00:00-23:57,False,FRADPE9147901818,2024-05-22,2024-05-22T15:33:49.735000+00:00
112,1922,support@alizecharge.fr,ADP Groupe,"[2.362554, 48.728787]","2 Rue des Transporteurs, 94390 Paray-Vieille-...",FRADPE914790181,ORY - Parking P3 SILO - Niveau 3,Parking public,16,Accès libre,Mo-Su 00:00-23:57,False,FRADPE9147901819,2024-05-22,2024-05-22T15:33:49.735000+00:00


In [167]:
itinerance_4.loc[itinerance_4.id_station_itinerance == 'FRADPE914790181', controles]

,index - id_pdc_itinerance,contact_operateur - id_station_itinerance,nom_enseigne - id_station_itinerance,coordonneesXY - id_station_itinerance,id_station_itinerance - id_pdc_itinerance,nom_station - id_station_itinerance,implantation_station - id_station_itinerance,nbre_pdc - id_station_itinerance,condition_acces - id_station_itinerance,horaires - id_station_itinerance,station_deux_roues - id_station_itinerance,adresse_station - coordonneesXY
103,True,True,True,True,True,False,False,True,True,True,True,False
104,True,True,True,True,True,False,False,True,True,True,True,False
105,True,True,True,True,True,False,False,True,True,True,True,False
106,True,True,True,True,True,False,False,True,True,True,True,False
107,True,True,True,True,True,False,False,True,True,True,True,False
108,True,True,True,True,True,False,False,True,True,True,True,False
109,True,True,True,True,True,False,False,True,True,True,True,False
110,True,True,True,True,True,False,False,True,True,True,True,False
111,True,True,True,True,True,False,False,True,True,True,True,False
112,True,True,True,True,True,False,False,True,True,True,True,False


In [163]:
itinerance_4.loc[itinerance_4.coordonneesXY == '[2.362554, 48.728787]', controles]

,index - id_pdc_itinerance,contact_operateur - id_station_itinerance,nom_enseigne - id_station_itinerance,coordonneesXY - id_station_itinerance,id_station_itinerance - id_pdc_itinerance,nom_station - id_station_itinerance,implantation_station - id_station_itinerance,nbre_pdc - id_station_itinerance,condition_acces - id_station_itinerance,horaires - id_station_itinerance,station_deux_roues - id_station_itinerance,adresse_station - coordonneesXY
102,True,True,True,True,True,True,True,True,True,True,True,False
103,True,True,True,True,True,False,False,True,True,True,True,False
104,True,True,True,True,True,False,False,True,True,True,True,False
105,True,True,True,True,True,False,False,True,True,True,True,False
106,True,True,True,True,True,False,False,True,True,True,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...
177,True,True,True,True,True,True,True,True,True,True,True,False
178,True,True,True,True,True,True,True,True,True,True,True,False
179,True,True,True,True,True,True,True,True,True,True,True,False
180,True,True,True,True,True,True,True,True,True,True,True,False


In [94]:
itinerance_4.loc[itinerance_4.coordonneesXY == '[-1.666373, 47.527509]', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
3819,56956,support@alizecharge.fr,Ouest Charge,"[-1.666373, 47.527509]","22 Route de l'Océan, 44170 LA GRIGONNAIS",FRS44E44224001,LA GRIGONNAIS - Route De L'Océan,Voirie,2,Accès libre,Mo-Su 00:00-23:57,False,FRS44E442240011,2024-05-22,2024-05-22T15:33:49.735000+00:00
3820,56957,support@alizecharge.fr,Ouest Charge,"[-1.666373, 47.527509]","22 Route de l'Océan, 44170 LA GRIGONNAIS",FRS44E44224001,LA GRIGONNAIS - Route De L'Océan,Voirie,2,Accès libre,Mo-Su 00:00-23:57,False,FRS44E442240012,2024-05-22,2024-05-22T15:33:49.735000+00:00
3821,56958,assistance-technique@ouestcharge-paysdelaloire...,SYDELA,"[-1.666373, 47.527509]",22 Route de l'Océan 44170 LA GRIGONNAIS (44),FRS44P44224A,OuestCharge - Diva Sp - La Grigonnais - Ocean,Voirie,1,Accès libre,24/7,false,FRS44E44224A1,2023-09-07,2024-01-19T07:46:12.032000+00:00
3822,56959,assistance-technique@ouestcharge-paysdelaloire...,SYDELA,"[-1.666373, 47.527509]",22 Route de l'Océan 44170 LA GRIGONNAIS (44),FRS44P44224A,OuestCharge - Diva Sp - La Grigonnais - Ocean,Voirie,1,Accès libre,24/7,false,FRS44E44224A2,2023-09-07,2024-01-19T07:46:12.032000+00:00


In [95]:
itinerance_4.loc[itinerance_4.coordonneesXY == '[-1.022143, 46.877453]', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
4018,62491,support@alizecharge.fr,SYDEV (FR*S85),"[-1.022143, 46.877453]","1 Avenue De L'Europe - Parking Origne Hall, 85...",FRS85E85109004,LES HERBIERS - Avenue De L'Europe - Parking Or...,Voirie,2,Accès libre,Mo-Su 00:00-23:57,False,FRS85E851090041,2024-05-22,2024-05-22T15:33:49.735000+00:00
4019,62492,support@alizecharge.fr,SYDEV (FR*S85),"[-1.022143, 46.877453]","1 Avenue De L'Europe - Parking Origne Hall, 85...",FRS85E85109004,LES HERBIERS - Avenue De L'Europe - Parking Or...,Voirie,2,Accès libre,Mo-Su 00:00-23:57,False,FRS85E851090042,2024-05-22,2024-05-22T15:33:49.735000+00:00
4020,62493,assistance-technique@ouestcharge-paysdelaloire...,SYDEV,"[-1.022143, 46.877453]",Avenue De L'Europe - Parking Origne Hall 85500...,FRS85P85109A,OuestCharge - 50 Compact - Les Herbiers - Europe,Voirie,2,Accès libre,24/7,false,FRS85E85109AB2CCS,2024-04-22,2024-04-22T09:38:03.113000+00:00
4021,62494,assistance-technique@ouestcharge-paysdelaloire...,SYDEV,"[-1.022143, 46.877453]",Avenue De L'Europe - Parking Origne Hall 85500...,FRS85P85109A,OuestCharge - 50 Compact - Les Herbiers - Europe,Voirie,2,Accès libre,24/7,false,FRS85E85109AB2CHADEMO,2024-04-22,2024-04-22T09:38:03.113000+00:00
4022,62495,assistance-technique@ouestcharge-paysdelaloire...,SYDEV,"[-1.022143, 46.877453]",Avenue De L'Europe - Parking Origne Hall 85500...,FRS85P85109A,OuestCharge - 50 Compact - Les Herbiers - Europe,Voirie,2,Accès libre,24/7,false,FRS85E85109AB2T2,2024-04-22,2024-04-22T09:38:03.113000+00:00


In [96]:
itinerance_4.loc[itinerance_4.coordonneesXY == '[2.362554, 48.728787]', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
102,1912,support@alizecharge.fr,ADP Groupe,"[2.362554, 48.728787]","2 Rue des Transporteurs , 94390 Paray-Vieille-...",FRADPE91479018,ORY - Parking P3 SILO - Niveau 2,Parking public,16,Accès libre,Mo-Su 00:00-23:57,False,FRADPE914790181,2024-05-22,2024-05-22T15:33:49.735000+00:00
103,1913,support@alizecharge.fr,ADP Groupe,"[2.362554, 48.728787]","2 Rue des Transporteurs , 94390 Paray-Vieille-...",FRADPE914790181,ORY - Parking P3 SILO - Niveau 2,Parking public,16,Accès libre,Mo-Su 00:00-23:57,False,FRADPE9147901810,2024-05-22,2024-05-22T15:33:49.735000+00:00
104,1914,support@alizecharge.fr,ADP Groupe,"[2.362554, 48.728787]","2 Rue des Transporteurs , 94390 Paray-Vieille-...",FRADPE914790181,ORY - Parking P3 SILO - Niveau 2,Parking public,16,Accès libre,Mo-Su 00:00-23:57,False,FRADPE9147901811,2024-05-22,2024-05-22T15:33:49.735000+00:00
105,1915,support@alizecharge.fr,ADP Groupe,"[2.362554, 48.728787]","2 Rue des Transporteurs , 94390 Paray-Vieille-...",FRADPE914790181,ORY - Parking P3 SILO - Niveau 2,Parking public,16,Accès libre,Mo-Su 00:00-23:57,False,FRADPE9147901812,2024-05-22,2024-05-22T15:33:49.735000+00:00
106,1916,support@alizecharge.fr,ADP Groupe,"[2.362554, 48.728787]","2 Rue des Transporteurs , 94390 Paray-Vieille-...",FRADPE914790181,ORY - Parking P3 SILO - Niveau 2,Parking public,16,Accès libre,Mo-Su 00:00-23:57,False,FRADPE9147901813,2024-05-22,2024-05-22T15:33:49.735000+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
177,1987,support@alizecharge.fr,ADP Groupe,"[2.362554, 48.728787]","2 Rue des Transporteurs , 94390 Paray-Vieille-...",FRADPE91479018,ORY - Parking P3 SILO - Niveau 2,Parking public,16,Accès libre,Mo-Su 00:00-23:57,False,FRADPE914790188,2024-05-22,2024-05-22T15:33:49.735000+00:00
178,1988,support@alizecharge.fr,ADP Groupe,"[2.362554, 48.728787]","2 Rue des Transporteurs, 94390 Paray-Vieille-P...",FRADPE914790188,ORY - Parking P3 SILO - Niveau 6,Parking public,18,Accès libre,Mo-Su 00:00-23:57,False,FRADPE9147901880,2024-05-22,2024-05-22T15:33:49.735000+00:00
179,1989,support@alizecharge.fr,ADP Groupe,"[2.362554, 48.728787]","2 Rue des Transporteurs, 94390 Paray-Vieille-P...",FRADPE914790188,ORY - Parking P3 SILO - Niveau 6,Parking public,18,Accès libre,Mo-Su 00:00-23:57,False,FRADPE9147901881,2024-05-22,2024-05-22T15:33:49.735000+00:00
180,1990,support@alizecharge.fr,ADP Groupe,"[2.362554, 48.728787]","2 Rue des Transporteurs, 94390 Paray-Vieille-P...",FRADPE914790188,ORY - Parking P3 SILO - Niveau 6,Parking public,18,Accès libre,Mo-Su 00:00-23:57,False,FRADPE9147901882,2024-05-22,2024-05-22T15:33:49.735000+00:00


### Defauts operateur IZIVIA
Les défauts au nombre de 280 sont liés à une incohérence entre adresse et coordonnées (une même coordonnée a plusieurs adresses différentes). Ce défaut est lié soit à des erreurs de saisie, soit à des ajouts (ou remplacements) ultérieurs de stations à une même localisation.

Les stations Izivia en erreur ont un id_station_itinerance identique à l'id_pdc_itinerance, ce qui se traduit par des adresses multiples pour une même coordonnée.

Par exemple, sur un parking [-0.530598, 47.398424], on trouve 17 stations avec la même coordonnée, le même identifiant entre point de recharge et station et un nombre de PdC indiqué entre 1 et 3.

In [97]:
itinerance_4.loc[itinerance_4['contact_operateur'] == 'sav@izivia.com', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
343,11345,sav@izivia.com,IZIVIA EXPRESS,"[-0.530598, 47.398424]",26 RUE VALENTIN DES ORMEAUX 49610 MURS-ERIGNE,FRE04POAZS24711,HYPER U - MURS-ERIGNE,Voirie,2,Accès libre,24/7,false,FRE04EOAZS24711,2024-06-04,2024-06-04T08:39:15.753000+00:00
344,11346,sav@izivia.com,IZIVIA EXPRESS,"[-0.530598, 47.398424]",26 RUE VALENTIN DES ORMEAUX 49610 MURS-ERIGNE,FRE04POAZS24712,HYPER U - MURS-ERIGNE,Voirie,2,Accès libre,24/7,false,FRE04EOAZS24712,2024-06-04,2024-06-04T08:39:15.753000+00:00
345,11347,sav@izivia.com,IZIVIA EXPRESS,"[-0.530598, 47.398424]",26 RUE VALENTIN DES ORMEAUX 49610 MURS-ERIGNE,FRE04POAZS24721,HYPER U - MURS-ERIGNE,Voirie,3,Accès libre,24/7,false,FRE04EOAZS24721,2024-06-04,2024-06-04T08:39:15.753000+00:00
346,11348,sav@izivia.com,IZIVIA EXPRESS,"[-0.530598, 47.398424]",26 RUE VALENTIN DES ORMEAUX 49610 MURS-ERIGNE,FRE04POAZS24722,HYPER U - MURS-ERIGNE,Voirie,3,Accès libre,24/7,false,FRE04EOAZS24722,2024-06-04,2024-06-04T08:39:15.753000+00:00
347,11349,sav@izivia.com,IZIVIA EXPRESS,"[-0.530598, 47.398424]",26 RUE VALENTIN DES ORMEAUX 49610 MURS-ERIGNE,FRE04POAZS24723,HYPER U - MURS-ERIGNE,Voirie,3,Accès libre,24/7,false,FRE04EOAZS24723,2024-06-04,2024-06-04T08:39:15.753000+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4411,74133,sav@izivia.com,UNIBAIL,"[2.523685, 48.9908]",CENTRE COMMERCIAL DAEROVILLE 95700 ROISSY-EN-F...,FRURWPUNIB1622,AEROVILLE - TERMINAL COOK,Voirie,2,Accès libre,24/7,false,FRURWEUNIB1622,2024-06-04,2024-06-04T08:39:14.401000+00:00
4412,74134,sav@izivia.com,UNIBAIL,"[2.523685, 48.9908]",CENTRE COMMERCIAL DAEROVILLE 95700 ROISSY-EN-F...,FRURWPUNIB1631,AEROVILLE - TERMINAL COOK,Voirie,2,Accès libre,24/7,false,FRURWEUNIB1631,2024-06-04,2024-06-04T08:39:14.401000+00:00
4413,74135,sav@izivia.com,UNIBAIL,"[2.523685, 48.9908]",CENTRE COMMERCIAL DAEROVILLE 95700 ROISSY-EN-F...,FRURWPUNIB1632,AEROVILLE - TERMINAL COOK,Voirie,2,Accès libre,24/7,false,FRURWEUNIB1632,2024-06-04,2024-06-04T08:39:14.401000+00:00
4414,74136,sav@izivia.com,UNIBAIL,"[2.523685, 48.9908]",CENTRE COMMERCIAL AEROVILLE 95700 ROISSY-EN-FR...,FRURWPUNIB1711,AEROVILLE - TOKYO,Voirie,2,Accès libre,24/7,false,FRURWEUNIB1711,2024-06-04,2024-06-04T08:39:14.401000+00:00


In [98]:
itinerance_4.loc[itinerance_4.coordonneesXY == '[-0.530598, 47.398424]', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
343,11345,sav@izivia.com,IZIVIA EXPRESS,"[-0.530598, 47.398424]",26 RUE VALENTIN DES ORMEAUX 49610 MURS-ERIGNE,FRE04POAZS24711,HYPER U - MURS-ERIGNE,Voirie,2,Accès libre,24/7,false,FRE04EOAZS24711,2024-06-04,2024-06-04T08:39:15.753000+00:00
344,11346,sav@izivia.com,IZIVIA EXPRESS,"[-0.530598, 47.398424]",26 RUE VALENTIN DES ORMEAUX 49610 MURS-ERIGNE,FRE04POAZS24712,HYPER U - MURS-ERIGNE,Voirie,2,Accès libre,24/7,false,FRE04EOAZS24712,2024-06-04,2024-06-04T08:39:15.753000+00:00
345,11347,sav@izivia.com,IZIVIA EXPRESS,"[-0.530598, 47.398424]",26 RUE VALENTIN DES ORMEAUX 49610 MURS-ERIGNE,FRE04POAZS24721,HYPER U - MURS-ERIGNE,Voirie,3,Accès libre,24/7,false,FRE04EOAZS24721,2024-06-04,2024-06-04T08:39:15.753000+00:00
346,11348,sav@izivia.com,IZIVIA EXPRESS,"[-0.530598, 47.398424]",26 RUE VALENTIN DES ORMEAUX 49610 MURS-ERIGNE,FRE04POAZS24722,HYPER U - MURS-ERIGNE,Voirie,3,Accès libre,24/7,false,FRE04EOAZS24722,2024-06-04,2024-06-04T08:39:15.753000+00:00
347,11349,sav@izivia.com,IZIVIA EXPRESS,"[-0.530598, 47.398424]",26 RUE VALENTIN DES ORMEAUX 49610 MURS-ERIGNE,FRE04POAZS24723,HYPER U - MURS-ERIGNE,Voirie,3,Accès libre,24/7,false,FRE04EOAZS24723,2024-06-04,2024-06-04T08:39:15.753000+00:00
2750,38731,sav@izivia.com,SYSTEME U,"[-0.530598, 47.398424]",RUE VALENTIN DES ORMEAUX 49610 MURS-ERIGNE,FROTHPOTHR372101,HYPER U - MURS-ERIGNE,Voirie,1,Accès libre,24/7,false,FROTHEOTHR372101,2024-06-04,2024-06-04T08:39:50.937000+00:00
2751,38732,sav@izivia.com,SYSTEME U,"[-0.530598, 47.398424]",RUE VALENTIN DES ORMEAUX 49610 MURS-ERIGNE,FROTHPOTHR37211,HYPER U - MURS-ERIGNE,Voirie,1,Accès libre,24/7,false,FROTHEOTHR37211,2024-06-04,2024-06-04T08:39:50.937000+00:00
2752,38733,sav@izivia.com,SYSTEME U,"[-0.530598, 47.398424]",RUE VALENTIN DES ORMEAUX 49610 MURS-ERIGNE,FROTHPOTHR372111,HYPER U - MURS-ERIGNE,Voirie,1,Accès libre,24/7,false,FROTHEOTHR372111,2024-06-04,2024-06-04T08:39:50.937000+00:00
2753,38734,sav@izivia.com,SYSTEME U,"[-0.530598, 47.398424]",RUE VALENTIN DES ORMEAUX 49610 MURS-ERIGNE,FROTHPOTHR372121,HYPER U - MURS-ERIGNE,Voirie,1,Accès libre,24/7,false,FROTHEOTHR372121,2024-06-04,2024-06-04T08:39:50.937000+00:00
2754,38735,sav@izivia.com,SYSTEME U,"[-0.530598, 47.398424]",RUE VALENTIN DES ORMEAUX 49610 MURS-ERIGNE,FROTHPOTHR37221,HYPER U - MURS-ERIGNE,Voirie,1,Accès libre,24/7,false,FROTHEOTHR37221,2024-06-04,2024-06-04T08:39:50.937000+00:00


### Defauts operateur IONITY
Les défauts (480 points de recharge) sont liés à une incohérence entre adresse et coordonnées (une même coordonnée a plusieurs adresses différentes). Ce défaut est lié soit à des erreurs de saisie, soit à des ajouts (ou remplacements) ultérieurs de stations à une même localisation.

Les stations Ionity en erreur ont un id_station_itinerance identique à l'id_pdc_itinerance ainsi qu'un ancien pdc de regroupement par station avec un nom d'enseigne différent (IONITY GMBH), ce qui se traduit par des adresses multiples pour une même coordonnée.

Par exemple, sur l'aire de Mornas [4.732409, 44.194832], on trouve 26 stations avec la même coordonnée, le même identifiant entre point de recharge et station et un nombre de PdC indiqué de 23 pour chaque station.

In [99]:
itinerance_4.loc[itinerance_4['contact_operateur'] == 'info@ionity.eu', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
1842,23446,info@ionity.eu,IONITY GMBH,"[0.366838, 46.701436]","Aire de Poitou-Charentes, A10, 79180 Vouillé",FRIONE124800,IONITY Poitou Charentes,Station dédiée à la recharge rapide,5,Accès libre,24/7,FALSE,FRIONE1248,2023-03-21,2024-01-19T07:47:22.735000+00:00
1843,23447,info@ionity.eu,IONITY GMBH,"[1.580008, 48.468782]","Aire de Chartres-Gasville, A11 - 28300 Gasvill...",FRIONE126400,IONITY Chartres Gasville,Station dédiée à la recharge rapide,5,Accès libre,24/7,FALSE,FRIONE1264,2023-03-21,2024-01-19T07:47:22.735000+00:00
1844,23448,info@ionity.eu,IONITY GMBH,"[1.578674, 48.467261]","Aire De Chartres Bois,A11 - 28300 Gasville-Oiséme",FRIONE126500,IONITY Chartres Bois Paris,Station dédiée à la recharge rapide,5,Accès libre,24/7,FALSE,FRIONE1265,2023-03-21,2024-01-19T07:47:22.735000+00:00
1845,23449,info@ionity.eu,IONITY GMBH,"[3.918671, 49.242974]","Aire de Vrigny,A4-51390 Vrigny",FRIONE401700,IONITY Vrigny,Station dédiée à la recharge rapide,5,Accès libre,24/7,FALSE,FRIONE4017,2023-03-21,2024-01-19T07:47:22.735000+00:00
1846,23452,info@ionity.eu,IONITY GMBH,"[1.416918, 46.313059]","Aire de Boismandé Ouest, A20 - 87160 Saint-Sul...",FRIONE402000,IONITY Boismandé Ouest,Station dédiée à la recharge rapide,5,Accès libre,24/7,FALSE,FRIONE4020,2023-03-21,2024-01-19T07:47:22.735000+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2265,24512,info@ionity.eu,Mornas Village,"[4.732409, 44.194832]","Aire de Mornas Village, A7, 84550 Mornas",FRIOYE440353,Mornas Village,Voirie,23,Accès libre,24/7,false,FRIOYE440353,2024-06-04,2024-06-05T01:00:09.736000+00:00
2266,24513,info@ionity.eu,Mornas Village,"[4.732409, 44.194832]","Aire de Mornas Village, A7, 84550 Mornas",FRIOYE440371,Mornas Village,Voirie,23,Accès libre,24/7,false,FRIOYE440371,2024-06-04,2024-06-05T01:00:09.736000+00:00
2267,24514,info@ionity.eu,Mornas Village,"[4.732409, 44.194832]","Aire de Mornas Village, A7, 84550 Mornas",FRIOYE440372,Mornas Village,Voirie,23,Accès libre,24/7,false,FRIOYE440372,2024-06-04,2024-06-05T01:00:09.736000+00:00
2268,24515,info@ionity.eu,Mornas Village,"[4.732409, 44.194832]","Aire de Mornas Village, A7, 84550 Mornas",FRIOYE440373,Mornas Village,Voirie,23,Accès libre,24/7,false,FRIOYE440373,2024-06-04,2024-06-05T01:00:09.736000+00:00


In [100]:
itinerance_4.loc[itinerance_4.coordonneesXY == '[1.580008, 48.468782]', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
1843,23447,info@ionity.eu,IONITY GMBH,"[1.580008, 48.468782]","Aire de Chartres-Gasville, A11 - 28300 Gasvill...",FRIONE126400,IONITY Chartres Gasville,Station dédiée à la recharge rapide,5,Accès libre,24/7,FALSE,FRIONE1264,2023-03-21,2024-01-19T07:47:22.735000+00:00
1890,23590,info@ionity.eu,Chartres Gasville,"[1.580008, 48.468782]","Chartres-Gasville A11, 28300 Gasville-Oisème",FRIOYE126401,Chartres Gasville,Voirie,7,Accès libre,24/7,false,FRIOYE126401,2024-06-04,2024-06-05T01:00:09.736000+00:00
1891,23591,info@ionity.eu,Chartres Gasville,"[1.580008, 48.468782]","Chartres-Gasville A11, 28300 Gasville-Oisème",FRIOYE126402,Chartres Gasville,Voirie,7,Accès libre,24/7,false,FRIOYE126402,2024-06-04,2024-06-05T01:00:09.736000+00:00
1892,23592,info@ionity.eu,Chartres Gasville,"[1.580008, 48.468782]","Chartres-Gasville A11, 28300 Gasville-Oisème",FRIOYE126404,Chartres Gasville,Voirie,7,Accès libre,24/7,false,FRIOYE126404,2024-06-04,2024-06-05T01:00:09.736000+00:00
1893,23593,info@ionity.eu,Chartres Gasville,"[1.580008, 48.468782]","Chartres-Gasville A11, 28300 Gasville-Oisème",FRIOYE126405,Chartres Gasville,Voirie,7,Accès libre,24/7,false,FRIOYE126405,2024-06-04,2024-06-05T01:00:09.736000+00:00
1894,23594,info@ionity.eu,Chartres Gasville,"[1.580008, 48.468782]","Chartres-Gasville A11, 28300 Gasville-Oisème",FRIOYE126451,Chartres Gasville,Voirie,7,Accès libre,24/7,false,FRIOYE126451,2024-06-04,2024-06-05T01:00:09.736000+00:00
1895,23595,info@ionity.eu,Chartres Gasville,"[1.580008, 48.468782]","Chartres-Gasville A11, 28300 Gasville-Oisème",FRIOYE126452,Chartres Gasville,Voirie,7,Accès libre,24/7,false,FRIOYE126452,2024-06-04,2024-06-05T01:00:09.736000+00:00
1896,23596,info@ionity.eu,Chartres Gasville,"[1.580008, 48.468782]","Chartres-Gasville A11, 28300 Gasville-Oisème",FRIOYE126453,Chartres Gasville,Voirie,7,Accès libre,24/7,false,FRIOYE126453,2024-06-04,2024-06-05T01:00:09.736000+00:00


In [101]:
itinerance_4.loc[itinerance_4.coordonneesXY == '[4.732409, 44.194832]', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
1888,23553,info@ionity.eu,IONITY GMBH,"[4.732409, 44.194832]","Aire de Mornas Village , A7, 84550 Mornas",FRIONE440300,IONITY Mornas Village,Station dédiée à la recharge rapide,16,Accès libre,24/7,FALSE,FRIONE4403,2023-03-21,2024-01-19T07:47:22.735000+00:00
2247,24494,info@ionity.eu,Mornas Village,"[4.732409, 44.194832]","Aire de Mornas Village, A7, 84550 Mornas",FRIOYE440301,Mornas Village,Voirie,23,Accès libre,24/7,false,FRIOYE440301,2024-06-04,2024-06-05T01:00:09.736000+00:00
2248,24495,info@ionity.eu,Mornas Village,"[4.732409, 44.194832]","Aire de Mornas Village, A7, 84550 Mornas",FRIOYE440302,Mornas Village,Voirie,23,Accès libre,24/7,false,FRIOYE440302,2024-06-04,2024-06-05T01:00:09.736000+00:00
2249,24496,info@ionity.eu,Mornas Village,"[4.732409, 44.194832]","Aire de Mornas Village, A7, 84550 Mornas",FRIOYE440303,Mornas Village,Voirie,23,Accès libre,24/7,false,FRIOYE440303,2024-06-04,2024-06-05T01:00:09.736000+00:00
2250,24497,info@ionity.eu,Mornas Village,"[4.732409, 44.194832]","Aire de Mornas Village, A7, 84550 Mornas",FRIOYE440304,Mornas Village,Voirie,23,Accès libre,24/7,false,FRIOYE440304,2024-06-04,2024-06-05T01:00:09.736000+00:00
2251,24498,info@ionity.eu,Mornas Village,"[4.732409, 44.194832]","Aire de Mornas Village, A7, 84550 Mornas",FRIOYE440305,Mornas Village,Voirie,23,Accès libre,24/7,false,FRIOYE440305,2024-06-04,2024-06-05T01:00:09.736000+00:00
2252,24499,info@ionity.eu,Mornas Village,"[4.732409, 44.194832]","Aire de Mornas Village, A7, 84550 Mornas",FRIOYE440306,Mornas Village,Voirie,23,Accès libre,24/7,false,FRIOYE440306,2024-06-04,2024-06-05T01:00:09.736000+00:00
2253,24500,info@ionity.eu,Mornas Village,"[4.732409, 44.194832]","Aire de Mornas Village, A7, 84550 Mornas",FRIOYE440307,Mornas Village,Voirie,23,Accès libre,24/7,false,FRIOYE440307,2024-06-04,2024-06-05T01:00:09.736000+00:00
2254,24501,info@ionity.eu,Mornas Village,"[4.732409, 44.194832]","Aire de Mornas Village, A7, 84550 Mornas",FRIOYE440308,Mornas Village,Voirie,23,Accès libre,24/7,false,FRIOYE440308,2024-06-04,2024-06-05T01:00:09.736000+00:00
2255,24502,info@ionity.eu,Mornas Village,"[4.732409, 44.194832]","Aire de Mornas Village, A7, 84550 Mornas",FRIOYE440309,Mornas Village,Voirie,23,Accès libre,24/7,false,FRIOYE440309,2024-06-04,2024-06-05T01:00:09.736000+00:00


In [102]:
itinerance_4.loc[itinerance_4['contact_operateur'] == 'info@ionity.eu', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
1842,23446,info@ionity.eu,IONITY GMBH,"[0.366838, 46.701436]","Aire de Poitou-Charentes, A10, 79180 Vouillé",FRIONE124800,IONITY Poitou Charentes,Station dédiée à la recharge rapide,5,Accès libre,24/7,FALSE,FRIONE1248,2023-03-21,2024-01-19T07:47:22.735000+00:00
1843,23447,info@ionity.eu,IONITY GMBH,"[1.580008, 48.468782]","Aire de Chartres-Gasville, A11 - 28300 Gasvill...",FRIONE126400,IONITY Chartres Gasville,Station dédiée à la recharge rapide,5,Accès libre,24/7,FALSE,FRIONE1264,2023-03-21,2024-01-19T07:47:22.735000+00:00
1844,23448,info@ionity.eu,IONITY GMBH,"[1.578674, 48.467261]","Aire De Chartres Bois,A11 - 28300 Gasville-Oiséme",FRIONE126500,IONITY Chartres Bois Paris,Station dédiée à la recharge rapide,5,Accès libre,24/7,FALSE,FRIONE1265,2023-03-21,2024-01-19T07:47:22.735000+00:00
1845,23449,info@ionity.eu,IONITY GMBH,"[3.918671, 49.242974]","Aire de Vrigny,A4-51390 Vrigny",FRIONE401700,IONITY Vrigny,Station dédiée à la recharge rapide,5,Accès libre,24/7,FALSE,FRIONE4017,2023-03-21,2024-01-19T07:47:22.735000+00:00
1846,23452,info@ionity.eu,IONITY GMBH,"[1.416918, 46.313059]","Aire de Boismandé Ouest, A20 - 87160 Saint-Sul...",FRIONE402000,IONITY Boismandé Ouest,Station dédiée à la recharge rapide,5,Accès libre,24/7,FALSE,FRIONE4020,2023-03-21,2024-01-19T07:47:22.735000+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2265,24512,info@ionity.eu,Mornas Village,"[4.732409, 44.194832]","Aire de Mornas Village, A7, 84550 Mornas",FRIOYE440353,Mornas Village,Voirie,23,Accès libre,24/7,false,FRIOYE440353,2024-06-04,2024-06-05T01:00:09.736000+00:00
2266,24513,info@ionity.eu,Mornas Village,"[4.732409, 44.194832]","Aire de Mornas Village, A7, 84550 Mornas",FRIOYE440371,Mornas Village,Voirie,23,Accès libre,24/7,false,FRIOYE440371,2024-06-04,2024-06-05T01:00:09.736000+00:00
2267,24514,info@ionity.eu,Mornas Village,"[4.732409, 44.194832]","Aire de Mornas Village, A7, 84550 Mornas",FRIOYE440372,Mornas Village,Voirie,23,Accès libre,24/7,false,FRIOYE440372,2024-06-04,2024-06-05T01:00:09.736000+00:00
2268,24515,info@ionity.eu,Mornas Village,"[4.732409, 44.194832]","Aire de Mornas Village, A7, 84550 Mornas",FRIOYE440373,Mornas Village,Voirie,23,Accès libre,24/7,false,FRIOYE440373,2024-06-04,2024-06-05T01:00:09.736000+00:00


In [103]:
itinerance_4.loc[itinerance_4.coordonneesXY == '[0.366838, 46.701436]', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
1842,23446,info@ionity.eu,IONITY GMBH,"[0.366838, 46.701436]","Aire de Poitou-Charentes, A10, 79180 Vouillé",FRIONE124800,IONITY Poitou Charentes,Station dédiée à la recharge rapide,5,Accès libre,24/7,FALSE,FRIONE1248,2023-03-21,2024-01-19T07:47:22.735000+00:00
2062,23921,info@ionity.eu,Poitiers Jaunay-Clan,"[0.366838, 46.701436]","Autoroute A10, 86130 Jaunay-Clan",FRIOYE409401,Poitiers Jaunay-Clan,Voirie,7,Accès libre,24/7,false,FRIOYE409401,2024-06-04,2024-06-05T01:00:09.736000+00:00
2063,23922,info@ionity.eu,Poitiers Jaunay-Clan,"[0.366838, 46.701436]","Autoroute A10, 86130 Jaunay-Clan",FRIOYE409402,Poitiers Jaunay-Clan,Voirie,7,Accès libre,24/7,false,FRIOYE409402,2024-06-04,2024-06-05T01:00:09.736000+00:00
2064,23923,info@ionity.eu,Poitiers Jaunay-Clan,"[0.366838, 46.701436]","Autoroute A10, 86130 Jaunay-Clan",FRIOYE409403,Poitiers Jaunay-Clan,Voirie,7,Accès libre,24/7,false,FRIOYE409403,2024-06-04,2024-06-05T01:00:09.736000+00:00
2065,23924,info@ionity.eu,Poitiers Jaunay-Clan,"[0.366838, 46.701436]","Autoroute A10, 86130 Jaunay-Clan",FRIOYE409404,Poitiers Jaunay-Clan,Voirie,7,Accès libre,24/7,false,FRIOYE409404,2024-06-04,2024-06-05T01:00:09.736000+00:00
2066,23925,info@ionity.eu,Poitiers Jaunay-Clan,"[0.366838, 46.701436]","Autoroute A10, 86130 Jaunay-Clan",FRIOYE409451,Poitiers Jaunay-Clan,Voirie,7,Accès libre,24/7,false,FRIOYE409451,2024-06-04,2024-06-05T01:00:09.736000+00:00
2067,23926,info@ionity.eu,Poitiers Jaunay-Clan,"[0.366838, 46.701436]","Autoroute A10, 86130 Jaunay-Clan",FRIOYE409452,Poitiers Jaunay-Clan,Voirie,7,Accès libre,24/7,false,FRIOYE409452,2024-06-04,2024-06-05T01:00:09.736000+00:00
2068,23927,info@ionity.eu,Poitiers Jaunay-Clan,"[0.366838, 46.701436]","Autoroute A10, 86130 Jaunay-Clan",FRIOYE409453,Poitiers Jaunay-Clan,Voirie,7,Accès libre,24/7,false,FRIOYE409453,2024-06-04,2024-06-05T01:00:09.736000+00:00


In [104]:
itinerance_4.loc[itinerance_4.coordonneesXY == '[2.523685, 48.9908]', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
4396,74118,sav@izivia.com,UNIBAIL,"[2.523685, 48.9908]",CENTRE COMMERCIAL DAEROVILLE 95700 ROISSY-EN-F...,FRURWPUNIB1311,AEROVILLE - AFRICA LODGE,Voirie,2,Accès libre,24/7,false,FRURWEUNIB1311,2024-06-04,2024-06-04T08:39:14.401000+00:00
4397,74119,sav@izivia.com,UNIBAIL,"[2.523685, 48.9908]",CENTRE COMMERCIAL DAEROVILLE 95700 ROISSY-EN-F...,FRURWPUNIB1312,AEROVILLE - AFRICA LODGE,Voirie,2,Accès libre,24/7,false,FRURWEUNIB1312,2024-06-04,2024-06-04T08:39:14.401000+00:00
4398,74120,sav@izivia.com,UNIBAIL,"[2.523685, 48.9908]",CENTRE COMMERCIAL DAEROVILLE 95700 ROISSY-EN-F...,FRURWPUNIB1321,AEROVILLE - AFRICA LODGE,Voirie,2,Accès libre,24/7,false,FRURWEUNIB1321,2024-06-04,2024-06-04T08:39:14.401000+00:00
4399,74121,sav@izivia.com,UNIBAIL,"[2.523685, 48.9908]",CENTRE COMMERCIAL DAEROVILLE 95700 ROISSY-EN-F...,FRURWPUNIB1322,AEROVILLE - AFRICA LODGE,Voirie,2,Accès libre,24/7,false,FRURWEUNIB1322,2024-06-04,2024-06-04T08:39:14.401000+00:00
4400,74122,sav@izivia.com,UNIBAIL,"[2.523685, 48.9908]",CENTRE COMMERCIAL DAEROVILLE 95700 ROISSY-EN-F...,FRURWPUNIB1411,AEROVILLE - BALI MARKET,Voirie,2,Accès libre,24/7,false,FRURWEUNIB1411,2024-06-04,2024-06-04T08:39:14.401000+00:00
4401,74123,sav@izivia.com,UNIBAIL,"[2.523685, 48.9908]",CENTRE COMMERCIAL DAEROVILLE 95700 ROISSY-EN-F...,FRURWPUNIB1412,AEROVILLE - BALI MARKET,Voirie,2,Accès libre,24/7,false,FRURWEUNIB1412,2024-06-04,2024-06-04T08:39:14.401000+00:00
4402,74124,sav@izivia.com,UNIBAIL,"[2.523685, 48.9908]",CENTRE COMMERCIAL DAEROVILLE 95700 ROISSY-EN-F...,FRURWPUNIB1421,AEROVILLE - BALI MARKET,Voirie,2,Accès libre,24/7,false,FRURWEUNIB1421,2024-06-04,2024-06-04T08:39:14.401000+00:00
4403,74125,sav@izivia.com,UNIBAIL,"[2.523685, 48.9908]",CENTRE COMMERCIAL DAEROVILLE 95700 ROISSY-EN-F...,FRURWPUNIB1422,AEROVILLE - BALI MARKET,Voirie,2,Accès libre,24/7,false,FRURWEUNIB1422,2024-06-04,2024-06-04T08:39:14.401000+00:00
4404,74126,sav@izivia.com,UNIBAIL,"[2.523685, 48.9908]",CENTRE COMMERCIAL DAEROVILLE 95700 ROISSY-EN-F...,FRURWPUNIB1511,AEROVILLE - NORDIC CHIC,Voirie,2,Accès libre,24/7,false,FRURWEUNIB1511,2024-06-04,2024-06-04T08:39:14.401000+00:00
4405,74127,sav@izivia.com,UNIBAIL,"[2.523685, 48.9908]",CENTRE COMMERCIAL DAEROVILLE 95700 ROISSY-EN-F...,FRURWPUNIB1512,AEROVILLE - NORDIC CHIC,Voirie,2,Accès libre,24/7,false,FRURWEUNIB1512,2024-06-04,2024-06-04T08:39:14.401000+00:00


### Cohérence implantation_station - id_station
- 107 pdc sont liés à une erreur de choix d'implantation (couplé à d'autres erreurs)

In [105]:
itinerance_4.loc[~itinerance_4['implantation_station - id_station_itinerance'], relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
43,377,contact@e55c.com,ELECTRIC 55 CHARGING,"[7.130067, 43.641297]",COOLWORK - VILLENEUVE-LOUBET,FR55CP06270VNLCOOLWORK,COOLWORK - VILLENEUVE-LOUBET,Parking privé à usage public,4,Accès réservé,24/7,FALSE,FR55CE062704364129771300611,2022-11-03,2024-05-16T11:29:02.272000+00:00
44,378,contact@e55c.com,ELECTRIC 55 CHARGING,"[7.130067, 43.641297]",COOLWORK - VILLENEUVE-LOUBET,FR55CP06270VNLCOOLWORK,COOLWORK - VILLENEUVE-LOUBET,Voirie,4,Accès réservé,24/7,FALSE,FR55CE062704364129771300621,2022-11-03,2024-05-16T11:29:02.272000+00:00
45,379,contact@e55c.com,ELECTRIC 55 CHARGING,"[7.130067, 43.641297]",COOLWORK - VILLENEUVE-LOUBET,FR55CP06270VNLCOOLWORK,COOLWORK - VILLENEUVE-LOUBET,Parking privé à usage public,4,Accès réservé,24/7,FALSE,FR55CE062704364129771300631,2022-11-03,2024-05-16T11:29:02.272000+00:00
46,380,contact@e55c.com,ELECTRIC 55 CHARGING,"[7.130067, 43.641297]",COOLWORK - VILLENEUVE-LOUBET,FR55CP06270VNLCOOLWORK,COOLWORK - VILLENEUVE-LOUBET,Parking privé à usage public,4,Accès réservé,24/7,FALSE,FR55CE062704364129771300641,2022-11-03,2024-05-16T11:29:02.272000+00:00
66,740,contact@e55c.com,ELECTRIC 55 CHARGING,"[5.976272, 45.526596]",BP - AIRE DE L'ABIS - SAINT-JEOIRE-PRIEURE - 7...,FR55CPBP73190AB1S,BP - AIRE DE L'ABIS - SAINT-JEOIRE-PRIEURE - 7...,Voirie,3,Accès réservé,24/7,FALSE,FR55CEFR7319043AB1S0,2022-11-03,2024-05-16T11:29:02.272000+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4202,65377,customerservice@shellrecharge.com,Shell Fenioux Est,"[-0.60405, 45.8923]","A10 Aire de Fenioux Est, 17350 Fenioux",FRSHEE94,Shell Fenioux Est,Voirie,8,Accès réservé,24/7,false,FRSHEE944,2024-06-04,2024-06-05T01:00:09.736000+00:00
4203,65378,customerservice@shellrecharge.com,Shell Fenioux Est,"[-0.60405, 45.8923]","A10 Aire de Fenioux Est, 17350 Fenioux",FRSHEE95,Shell Fenioux Est,Station dédiée à la recharge rapide,8,Accès réservé,24/7,false,FRSHEE951,2024-06-04,2024-06-05T01:00:09.736000+00:00
4204,65379,customerservice@shellrecharge.com,Shell Fenioux Est,"[-0.60405, 45.8923]","A10 Aire de Fenioux Est, 17350 Fenioux",FRSHEE95,Shell Fenioux Est,Station dédiée à la recharge rapide,8,Accès réservé,24/7,false,FRSHEE952,2024-06-04,2024-06-05T01:00:09.736000+00:00
4205,65380,customerservice@shellrecharge.com,Shell Fenioux Est,"[-0.60405, 45.8923]","A10 Aire de Fenioux Est, 17350 Fenioux",FRSHEE95,Shell Fenioux Est,Station dédiée à la recharge rapide,8,Accès réservé,24/7,false,FRSHEE953,2024-06-04,2024-06-05T01:00:09.736000+00:00


### Cohérence nom_station - id_station
- 210 pdc sont associés à une station avec un nom non cohérent

In [106]:
itinerance_4.loc[~itinerance_4['nom_station - id_station_itinerance'], relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
66,740,contact@e55c.com,ELECTRIC 55 CHARGING,"[5.976272, 45.526596]",BP - AIRE DE L'ABIS - SAINT-JEOIRE-PRIEURE - 7...,FR55CPBP73190AB1S,BP - AIRE DE L'ABIS - SAINT-JEOIRE-PRIEURE - 7...,Voirie,3,Accès réservé,24/7,FALSE,FR55CEFR7319043AB1S0,2022-11-03,2024-05-16T11:29:02.272000+00:00
67,741,contact@e55c.com,ELECTRIC 55 CHARGING,"[5.976272, 45.526596]",BP - AIRE DE L'ABIS - SAINT-JEOIRE-PRIEURE - 7...,FR55CPBP73190AB1S,BP - AIRE DE L'ABIS - SAINT-JEOIRE-PRIEURE - 7...,Voirie,3,Accès réservé,24/7,FALSE,FR55CEFR7319043AB1S1,2022-11-03,2024-05-16T11:29:02.272000+00:00
68,742,contact@e55c.com,ELECTRIC 55 CHARGING,"[5.976272, 45.526596]",BP - AIRE DE L'ABIS - SAINT-JEOIRE-PRIEURE - 7...,FR55CPBP73190AB1S,BP - AIRE DE L'ABIS - SAINT-JEOIRE-PRIEURE - 7...,Voirie,3,Accès réservé,24/7,FALSE,FR55CEFR7319043AB1S2,2022-11-03,2024-05-16T11:29:02.272000+00:00
69,743,contact@e55c.com,ELECTRIC 55 CHARGING,"[5.976272, 45.526596]",BP - AIRE DE L'ABIS SAINT-JEOIRE-PRIEURE,FR55CPBP73190AB1S,BP - AIRE DE L'ABIS SAINT-JEOIRE-PRIEURE - 73190,Station dédiée à la recharge rapide,8,Accès réservé,24/7,FALSE,FR55CEFR7319043AB1S3,2022-11-03,2024-05-16T11:29:02.272000+00:00
70,744,contact@e55c.com,ELECTRIC 55 CHARGING,"[5.976272, 45.526596]",BP - AIRE DE L'ABIS SAINT-JEOIRE-PRIEURE,FR55CPBP73190AB1S,BP - AIRE DE L'ABIS SAINT-JEOIRE-PRIEURE - 73190,Station dédiée à la recharge rapide,8,Accès réservé,24/7,FALSE,FR55CEFR7319043AB1S4,2022-11-03,2024-05-16T11:29:02.272000+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4319,71410,support@alizecharge.fr,CPO Alizé Liberté Public,"[1.4503666, 43.6039735]","61 Bd Lazare Carnot, 31000 TOULOUSE",FRTLSE31555040,TOULOUSE - Bd Lazare Carnot,Voirie,3,Accès libre,Mo-Su 00:00-23:57,False,FRTLSE315550401,2024-05-22,2024-05-22T15:33:49.735000+00:00
4320,71411,support@alizecharge.fr,CPO Alizé Liberté Public,"[1.4503666, 43.6039735]","61 Bd Lazare Carnot, 31000 TOULOUSE",FRTLSE31555040,TOULOUSE - Bd Lazare Carnot,Voirie,3,Accès libre,Mo-Su 00:00-23:57,False,FRTLSE315550402,2024-05-22,2024-05-22T15:33:49.735000+00:00
4321,71412,support@alizecharge.fr,CPO Alizé Liberté Public,"[1.45026, 43.6040701]","63 Bd Lazare Carnot, 31000 TOULOUSE",FRTLSE31555040,TOULOUSE – Station Deux-Roues Lazare Carnot,Voirie,3,Accès libre,Mo-Su 00:00-23:57,False,FRTLSE315550403,2024-05-22,2024-05-22T15:33:49.735000+00:00
4322,71413,support@alizecharge.fr,CPO Alizé Liberté Public,"[1.45026, 43.6040701]","63 Bd Lazare Carnot, 31000 TOULOUSE",FRTLSE31555040,TOULOUSE – Station Deux-Roues Lazare Carnot,Voirie,3,Accès libre,Mo-Su 00:00-23:57,False,FRTLSE315550404,2024-05-22,2024-05-22T15:33:49.735000+00:00


### Coherence adresse - coordonnees
- 2337 pdc ont une adresse non cohérente avec les coordonnées géographiques

In [107]:
itinerance_4.loc[~itinerance_4['adresse_station - coordonneesXY'], relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
19,160,exploitation@r3-charge.fr,R3,"[3.472424, 50.379911]","400 Rue Michel Chasles, 59494 Petite-Forêt",FR3R3P89616641,R3 - Petite Forêt,Parking privé à usage public,5,Accès libre,Mo-Su 07:00-22:00,false,FR3R3E10000849681,2024-03-29,2024-03-30T07:00:45.287000+00:00
20,161,exploitation@r3-charge.fr,R3,"[3.472424, 50.379911]","401 Rue Michel Chasles, 59494 Petite-Forêt",FR3R3P89616641,R3 - Petite Forêt,Parking privé à usage public,5,Accès libre,Mo-Su 07:00-22:01,false,FR3R3E10000849682,2024-03-29,2024-03-30T07:00:45.287000+00:00
21,162,exploitation@r3-charge.fr,R3,"[1.613217, 50.485562]",708 Av. François Godin Bis 62780 Cucq,FR3R3P89629589,R3 - Cucq,Parking privé à usage public,7,Accès libre,24/7,false,FR3R3E10000849691,2024-03-29,2024-03-30T07:00:45.287000+00:00
22,163,exploitation@r3-charge.fr,R3,"[1.613217, 50.485562]",709 Av. François Godin Bis 62780 Cucq,FR3R3P89629589,R3 - Cucq,Parking privé à usage public,7,Accès libre,24/7,false,FR3R3E10000849692,2024-03-29,2024-03-30T07:00:45.287000+00:00
23,186,exploitation@r3-charge.fr,R3,"[1.613217, 50.485562]",713 Av. François Godin Bis 62780 Cucq,FR3R3P89629589,R3 - Cucq,Parking privé à usage public,7,Accès libre,24/7,false,FR3R3E10000849851,2024-03-29,2024-03-30T07:00:45.287000+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4906,77566,advenir@zeborne.com,GARAGE NELLO CHELLI - DIJON,"[5.065799, 47.354481]",23 RUE DES ARDENNES 21000 Dijon,FRZTLE22AC55087,Toyota - Dijon - 22kW AC,Parking public,1,Accès libre,"Mo-Fr 07:45-12:00,Mo-Fr 13:45-19:00,Sat 09:00-...",false,FRZTLE22AC55087,2021-02-19,2024-01-19T07:46:24.141000+00:00
4907,77575,advenir@zeborne.com,TOYOTA MONTAGNAT - DJB BOURG,"[5.2592028, 46.1746523]",RN 75 ZI DE NOIREFONTAINE 01250 Montagnat,FRZTLE22AC57332,Toyota - Montagnat - 22kW AC,Parking public,1,Accès libre,"Mo-Fr 08:00–12:00,Mo-Fr 14:00–19:00,Sat 09:00-...",false,FRZTLE22AC57332,2021-04-08,2024-01-19T07:46:23.908000+00:00
4908,77576,advenir@zeborne.com,TOYOTA MONTAGNAT - DJB BOURG,"[5.2592028, 46.1746523]",RN 75 ZI DE NOIREFONTAINE 01250 Montagnat,FRZTLE22AC57333,Toyota - Montagnat - 22kW AC,Parking public,1,Accès libre,"Mo-Fr 08:00–12:00,Mo-Fr 14:00–19:00,Sat 09:00-...",false,FRZTLE22AC57333,2021-04-08,2024-01-19T07:46:23.908000+00:00
4909,77577,advenir@zeborne.com,TOYOTA MONTAGNAT - DJB BOURG,"[5.2592028, 46.1746523]",RN 75 ZI DE NOIREFONTAINE 01250 Montagnat,FRZTLE22AC57334,Toyota - Montagnat - 22kW AC,Parking public,1,Accès libre,"Mo-Fr 08:00–12:00,Mo-Fr 14:00–19:00,Sat 09:00-...",false,FRZTLE22AC57334,2021-04-08,2024-01-19T07:46:23.908000+00:00


In [108]:
itinerance_4.loc[itinerance_4.coordonneesXY == '[5.2592028, 46.1746523]', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
4907,77575,advenir@zeborne.com,TOYOTA MONTAGNAT - DJB BOURG,"[5.2592028, 46.1746523]",RN 75 ZI DE NOIREFONTAINE 01250 Montagnat,FRZTLE22AC57332,Toyota - Montagnat - 22kW AC,Parking public,1,Accès libre,"Mo-Fr 08:00–12:00,Mo-Fr 14:00–19:00,Sat 09:00-...",false,FRZTLE22AC57332,2021-04-08,2024-01-19T07:46:23.908000+00:00
4908,77576,advenir@zeborne.com,TOYOTA MONTAGNAT - DJB BOURG,"[5.2592028, 46.1746523]",RN 75 ZI DE NOIREFONTAINE 01250 Montagnat,FRZTLE22AC57333,Toyota - Montagnat - 22kW AC,Parking public,1,Accès libre,"Mo-Fr 08:00–12:00,Mo-Fr 14:00–19:00,Sat 09:00-...",false,FRZTLE22AC57333,2021-04-08,2024-01-19T07:46:23.908000+00:00
4909,77577,advenir@zeborne.com,TOYOTA MONTAGNAT - DJB BOURG,"[5.2592028, 46.1746523]",RN 75 ZI DE NOIREFONTAINE 01250 Montagnat,FRZTLE22AC57334,Toyota - Montagnat - 22kW AC,Parking public,1,Accès libre,"Mo-Fr 08:00–12:00,Mo-Fr 14:00–19:00,Sat 09:00-...",false,FRZTLE22AC57334,2021-04-08,2024-01-19T07:46:23.908000+00:00
4910,77578,advenir@zeborne.com,TOYOTA MONTAGNAT - DJB BOURG,"[5.2592028, 46.1746523]",RN 75 ZI DE NOIREFONTAINE 01250 Montagnat,FRZTLE22AC57335,Toyota - Montagnat - 22kW AC,Parking public,1,Accès libre,"Mo-Fr 08:00–12:00,Mo-Fr 14:00–19:00,Sat 09:00-...",false,FRZTLE22AC57335,2021-04-08,2024-01-19T07:46:23.908000+00:00


In [109]:
itinerance_4.loc[itinerance_4.adresse_station == 'RN 75 ZI DE NOIREFONTAINE 01250 Montagnat', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
4907,77575,advenir@zeborne.com,TOYOTA MONTAGNAT - DJB BOURG,"[5.2592028, 46.1746523]",RN 75 ZI DE NOIREFONTAINE 01250 Montagnat,FRZTLE22AC57332,Toyota - Montagnat - 22kW AC,Parking public,1,Accès libre,"Mo-Fr 08:00–12:00,Mo-Fr 14:00–19:00,Sat 09:00-...",false,FRZTLE22AC57332,2021-04-08,2024-01-19T07:46:23.908000+00:00
4909,77577,advenir@zeborne.com,TOYOTA MONTAGNAT - DJB BOURG,"[5.2592028, 46.1746523]",RN 75 ZI DE NOIREFONTAINE 01250 Montagnat,FRZTLE22AC57334,Toyota - Montagnat - 22kW AC,Parking public,1,Accès libre,"Mo-Fr 08:00–12:00,Mo-Fr 14:00–19:00,Sat 09:00-...",false,FRZTLE22AC57334,2021-04-08,2024-01-19T07:46:23.908000+00:00


In [110]:
itinerance_4.loc[itinerance_4.adresse_station == ' RN 75 ZI DE NOIREFONTAINE 01250 Montagnat', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
4908,77576,advenir@zeborne.com,TOYOTA MONTAGNAT - DJB BOURG,"[5.2592028, 46.1746523]",RN 75 ZI DE NOIREFONTAINE 01250 Montagnat,FRZTLE22AC57333,Toyota - Montagnat - 22kW AC,Parking public,1,Accès libre,"Mo-Fr 08:00–12:00,Mo-Fr 14:00–19:00,Sat 09:00-...",false,FRZTLE22AC57333,2021-04-08,2024-01-19T07:46:23.908000+00:00
4910,77578,advenir@zeborne.com,TOYOTA MONTAGNAT - DJB BOURG,"[5.2592028, 46.1746523]",RN 75 ZI DE NOIREFONTAINE 01250 Montagnat,FRZTLE22AC57335,Toyota - Montagnat - 22kW AC,Parking public,1,Accès libre,"Mo-Fr 08:00–12:00,Mo-Fr 14:00–19:00,Sat 09:00-...",false,FRZTLE22AC57335,2021-04-08,2024-01-19T07:46:23.908000+00:00


### Cohérence station - coordonnées
- 218 pdc sont associés à des stations avec plusieurs coordonnées (majorité AlizeCharge)

In [111]:
itinerance_4.loc[~itinerance_4['coordonneesXY - id_station_itinerance'], relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
0,6,acelec@acelec-france.com,Camaïeu France,"[3.207306,50.684918]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073E0HKH51125,2022-07-21,2024-01-19T07:46:04.014000+00:00
1,7,acelec@acelec-france.com,Camaïeu France,"[3.207246,50.68494]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073E8OV45243,2022-07-21,2024-01-19T07:46:04.014000+00:00
2,8,acelec@acelec-france.com,Camaïeu France,"[3.207124,50.684984]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073EBEQA5957,2022-07-21,2024-01-19T07:46:04.014000+00:00
3,9,acelec@acelec-france.com,Camaïeu France,"[3.20737,50.684898]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073ECC1C5530,2022-07-21,2024-01-19T07:46:04.014000+00:00
4,10,acelec@acelec-france.com,HOTEL KYRIAD PARIS BEZONS,"[2.212336,48.931356]",80 Av. Gabriel Péri 95870 BEZONS,FR073PKYRIADBEZONS,HOTEL KYRIAD PARIS BEZONS,Parking privé à usage public,1,Accès libre,Mo-Fr 08:00-19:01,false,FR073EJJYD51230,2022-07-21,2024-01-19T07:46:04.014000+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4319,71410,support@alizecharge.fr,CPO Alizé Liberté Public,"[1.4503666, 43.6039735]","61 Bd Lazare Carnot, 31000 TOULOUSE",FRTLSE31555040,TOULOUSE - Bd Lazare Carnot,Voirie,3,Accès libre,Mo-Su 00:00-23:57,False,FRTLSE315550401,2024-05-22,2024-05-22T15:33:49.735000+00:00
4320,71411,support@alizecharge.fr,CPO Alizé Liberté Public,"[1.4503666, 43.6039735]","61 Bd Lazare Carnot, 31000 TOULOUSE",FRTLSE31555040,TOULOUSE - Bd Lazare Carnot,Voirie,3,Accès libre,Mo-Su 00:00-23:57,False,FRTLSE315550402,2024-05-22,2024-05-22T15:33:49.735000+00:00
4321,71412,support@alizecharge.fr,CPO Alizé Liberté Public,"[1.45026, 43.6040701]","63 Bd Lazare Carnot, 31000 TOULOUSE",FRTLSE31555040,TOULOUSE – Station Deux-Roues Lazare Carnot,Voirie,3,Accès libre,Mo-Su 00:00-23:57,False,FRTLSE315550403,2024-05-22,2024-05-22T15:33:49.735000+00:00
4322,71413,support@alizecharge.fr,CPO Alizé Liberté Public,"[1.45026, 43.6040701]","63 Bd Lazare Carnot, 31000 TOULOUSE",FRTLSE31555040,TOULOUSE – Station Deux-Roues Lazare Carnot,Voirie,3,Accès libre,Mo-Su 00:00-23:57,False,FRTLSE315550404,2024-05-22,2024-05-22T15:33:49.735000+00:00


In [112]:
itinerance_5.loc[itinerance_5.id_station_itinerance	 == 'FRTLSE31555040', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
3026,71410,support@alizecharge.fr,CPO Alizé Liberté Public,"[1.4503666, 43.6039735]","61 Bd Lazare Carnot, 31000 TOULOUSE",FRTLSE31555040,TOULOUSE - Bd Lazare Carnot,Voirie,3,Accès libre,Mo-Su 00:00-23:57,False,FRTLSE315550401,2024-05-22,2024-05-22T15:33:49.735000+00:00
3027,71411,support@alizecharge.fr,CPO Alizé Liberté Public,"[1.4503666, 43.6039735]","61 Bd Lazare Carnot, 31000 TOULOUSE",FRTLSE31555040,TOULOUSE - Bd Lazare Carnot,Voirie,3,Accès libre,Mo-Su 00:00-23:57,False,FRTLSE315550402,2024-05-22,2024-05-22T15:33:49.735000+00:00
3028,71412,support@alizecharge.fr,CPO Alizé Liberté Public,"[1.45026, 43.6040701]","63 Bd Lazare Carnot, 31000 TOULOUSE",FRTLSE31555040,TOULOUSE – Station Deux-Roues Lazare Carnot,Voirie,3,Accès libre,Mo-Su 00:00-23:57,False,FRTLSE315550403,2024-05-22,2024-05-22T15:33:49.735000+00:00
3029,71413,support@alizecharge.fr,CPO Alizé Liberté Public,"[1.45026, 43.6040701]","63 Bd Lazare Carnot, 31000 TOULOUSE",FRTLSE31555040,TOULOUSE – Station Deux-Roues Lazare Carnot,Voirie,3,Accès libre,Mo-Su 00:00-23:57,False,FRTLSE315550404,2024-05-22,2024-05-22T15:33:49.735000+00:00
3030,71414,support@alizecharge.fr,CPO Alizé Liberté Public,"[1.45026, 43.6040701]","63 Bd Lazare Carnot, 31000 TOULOUSE",FRTLSE31555040,TOULOUSE – Station Deux-Roues Lazare Carnot,Voirie,3,Accès libre,Mo-Su 00:00-23:57,False,FRTLSE315550405,2024-05-22,2024-05-22T15:33:49.735000+00:00


In [113]:
itinerance_5.loc[itinerance_5.id_station_itinerance	 == 'FRFR1PEHNKRRPQNR', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified


- exemple : une station avec plusieurs coordonnées -> incompréhension distinction station / pdc

In [114]:
itinerance_5.loc[itinerance_5.id_station_itinerance == 'FR073PCAMAIEUFR', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
0,6,acelec@acelec-france.com,Camaïeu France,"[3.207306,50.684918]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073E0HKH51125,2022-07-21,2024-01-19T07:46:04.014000+00:00
1,7,acelec@acelec-france.com,Camaïeu France,"[3.207246,50.68494]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073E8OV45243,2022-07-21,2024-01-19T07:46:04.014000+00:00
2,8,acelec@acelec-france.com,Camaïeu France,"[3.207124,50.684984]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073EBEQA5957,2022-07-21,2024-01-19T07:46:04.014000+00:00
3,9,acelec@acelec-france.com,Camaïeu France,"[3.20737,50.684898]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073ECC1C5530,2022-07-21,2024-01-19T07:46:04.014000+00:00
5,11,acelec@acelec-france.com,Camaïeu France,"[3.207306,50.684918]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073EK0BJ522,2022-07-21,2024-01-19T07:46:04.014000+00:00
6,12,acelec@acelec-france.com,Camaïeu France,"[3.207047,50.685013]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073EMZLV5283,2022-07-21,2024-01-19T07:46:04.014000+00:00
7,13,acelec@acelec-france.com,Camaïeu France,"[3.20737,50.684898]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073EO0AV51091,2022-07-21,2024-01-19T07:46:04.014000+00:00
8,14,acelec@acelec-france.com,Camaïeu France,"[3.207187,50.684964]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073EP7QU5233,2022-07-21,2024-01-19T07:46:04.014000+00:00
9,15,acelec@acelec-france.com,Camaïeu France,"[3.206962,50.685049]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073EP9LD51287,2022-07-21,2024-01-19T07:46:04.014000+00:00
10,16,acelec@acelec-france.com,Camaïeu France,"[3.207433,50.684876]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073EQ1GI51176,2022-07-21,2024-01-19T07:46:04.014000+00:00


- exemple : une station avec plusieurs coordonnées -> regroupement de stations ?

In [115]:
itinerance_5.loc[itinerance_5.id_station_itinerance == 'FR55CPBP514', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified


- exemple : plusieurs stations d'opérateurs différents avec les mêmes coordonnées -> suppression non effectuée lors d'un changement d'opérateur ?

In [116]:
itinerance_4.loc[itinerance_4.coordonneesXY == '[0.08745032, 48.40887156]', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified


In [117]:
itinerance_4.loc[itinerance_4.coordonneesXY == '[-4.374469, 48.471878]', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified


- exemple : quatre stations avec une localisation identique mais deux adresses différentes + id identique entre station et pdc -> erreur d'adresse et erreur d'id_station ?

In [118]:
itinerance_5.loc[itinerance_5.coordonneesXY	 == '[-0.366184, 43.332508]', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified


### Incoherence nbre_pdc
- 583 pdc ont un champ 'nbre_pdc' mal documenté

In [119]:
itinerance_4.loc[~itinerance_4['nbre_pdc - id_station_itinerance'], relations][200:215]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
539,17968,contact@e-totem.fr,INSEAD Fontainebleau,"[2.683842208923085, 48.40464956224202]",Bd de Constance 77300 Fontainebleau,FRG10P77186A,INSEAD Fontainebleau,Parking privé à usage public,10,Accès libre,24/7,false,FRG10E77186A12,2024-03-13,2024-03-14T07:13:52.595000+00:00
540,17969,contact@e-totem.fr,INSEAD Fontainebleau,"[2.683842208923085, 48.40464956224202]",Bd de Constance 77300 Fontainebleau,FRG10P77186A,INSEAD Fontainebleau,Parking privé à usage public,10,Accès libre,24/7,false,FRG10E77186A13,2024-03-13,2024-03-14T07:13:52.595000+00:00
541,17970,contact@e-totem.fr,INSEAD Fontainebleau,"[2.683842208923085, 48.40464956224202]",Bd de Constance 77300 Fontainebleau,FRG10P77186A,INSEAD Fontainebleau,Parking privé à usage public,10,Accès libre,24/7,false,FRG10E77186A14,2024-03-13,2024-03-14T07:13:52.595000+00:00
542,17971,contact@e-totem.fr,INSEAD Fontainebleau,"[2.683842208923085, 48.40464956224202]",Bd de Constance 77300 Fontainebleau,FRG10P77186A,INSEAD Fontainebleau,Parking privé à usage public,10,Accès libre,24/7,false,FRG10E77186A15,2024-03-13,2024-03-14T07:13:52.595000+00:00
543,17972,contact@e-totem.fr,INSEAD Fontainebleau,"[2.683842208923085, 48.40464956224202]",Bd de Constance 77300 Fontainebleau,FRG10P77186A,INSEAD Fontainebleau,Parking privé à usage public,10,Accès libre,24/7,false,FRG10E77186A16,2024-03-13,2024-03-14T07:13:52.595000+00:00
544,17973,contact@e-totem.fr,INSEAD Fontainebleau,"[2.683842208923085, 48.40464956224202]",Bd de Constance 77300 Fontainebleau,FRG10P77186A,INSEAD Fontainebleau,Parking privé à usage public,10,Accès libre,24/7,false,FRG10E77186A17,2024-03-13,2024-03-14T07:13:52.595000+00:00
545,17974,contact@e-totem.fr,INSEAD Fontainebleau,"[2.683842208923085, 48.40464956224202]",Bd de Constance 77300 Fontainebleau,FRG10P77186A,INSEAD Fontainebleau,Parking privé à usage public,10,Accès libre,24/7,false,FRG10E77186A18,2024-03-13,2024-03-14T07:13:52.595000+00:00
546,17975,contact@e-totem.fr,INSEAD Fontainebleau,"[2.683842208923085, 48.40464956224202]",Bd de Constance 77300 Fontainebleau,FRG10P77186A,INSEAD Fontainebleau,Parking privé à usage public,10,Accès libre,24/7,false,FRG10E77186A19,2024-03-13,2024-03-14T07:13:52.595000+00:00
547,17976,contact@e-totem.fr,INSEAD Fontainebleau,"[2.683842208923085, 48.40464956224202]",Bd de Constance 77300 Fontainebleau,FRG10P77186A,INSEAD Fontainebleau,Parking privé à usage public,5,Accès libre,24/7,false,FRG10E77186A21,2024-03-13,2024-03-14T07:13:52.595000+00:00
548,17977,contact@e-totem.fr,INSEAD Fontainebleau,"[2.683842208923085, 48.40464956224202]",Bd de Constance 77300 Fontainebleau,FRG10P77186A,INSEAD Fontainebleau,Parking privé à usage public,5,Accès libre,24/7,false,FRG10E77186A22,2024-03-13,2024-03-14T07:13:52.595000+00:00


### coherence station - enseigne
- 5 stations ont plusieurs noms d'enseigne

In [120]:
itinerance_4.loc[~itinerance_4['nom_enseigne - id_station_itinerance'], relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
448,17703,contact@electricite-loos.fr,RME Loos (FR-FR1),"[3.009613, 50.617509]",12 Rue Salengro LOOS,FRFR1EPJXS,"Loos, Parking CIL",Parking public,3,Accès libre,24/7,FALSE,FRFR1EPJXS1,2022-10-04,2024-01-19T07:49:31.469000+00:00
449,17704,contact@electricite-loos.fr,RME Loos (FR-FR1),"[3.009613, 50.617509]",12 Rue Salengro LOOS,FRFR1EPJXS,"Loos, Parking CIL",Parking public,3,Accès libre,24/7,FALSE,FRFR1EPJXS2,2022-10-04,2024-01-19T07:49:31.469000+00:00
450,17705,contact@electricite-loos.fr,RME Loos (FR-FR1),"[3.009613, 50.617509]",12 Rue Salengro LOOS,FRFR1EPJXS,"Loos, Parking CIL",Parking public,3,Accès libre,24/7,FALSE,FRFR1EPJXS3,2022-10-04,2024-01-19T07:49:31.469000+00:00
2412,31533,contact@rirodo.fr,LAST MILES SOLUTIONS,"[5.39, 43.27]",20 Rue Negresko,FRRIRE2804856,PARKING NEGRESKO,Parking privé à usage public,2,Accès libre,24/7,false,FRLMSE1897228,2023-09-06,2024-01-19T07:49:16.739000+00:00
2916,51175,contact@rirodo.fr,MONTA,"[5.39, 43.27]",20 Rue NEGRESKO,FRRIRE2804856,PARKING NEGRESKO,Parking privé à usage public,2,Accès libre,24/7,false,FRRIRE2804856,2023-09-06,2024-01-19T07:49:16.739000+00:00


In [121]:
itinerance_init.loc[itinerance_init.id_station_itinerance == 'FRFR1EAYRU', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
17660,17678,contact@electricite-loos.fr,RME Loos (FR-FR1),"[3.02,50.61]",rue du colonnel jean colonna d'ornano loos,FRFR1EAYRU,"Loos, Parking de la Gare",Parking public,3,Accès libre,24/7,false,FRFR1EAYRU1,2022-12-25,2024-01-19T07:49:57.766000+00:00
17661,17679,contact@electricite-loos.fr,RME Loos (FR-FR1),"[3.02,50.61]",rue du colonnel ornano loos,FRFR1EAYRU,"Loos, Parking de la Gare",Parking public,3,Accès libre,24/7,false,FRFR1EAYRU2,2022-08-25,2024-01-19T07:49:57.766000+00:00
17662,17680,contact@electricite-loos.fr,RME Loos (FR-FR1),"[3.02,50.61]",rue du colonnel ornano loos,FRFR1EAYRU,"Loos, Parking de la Gare",Parking public,3,Accès libre,24/7,false,FRFR1EAYRU3,2022-08-25,2024-01-19T07:49:57.766000+00:00
17669,17687,contact@electricite-loos.fr,RME Loos (FR-FR1),"[3.014273, 50.615449]",12 Rue Salengro LOOS,FRFR1EAYRU,"Loos, Parking Salengro",Parking public,3,Accès libre,24/7,FALSE,FRFR1EDXS1,2022-12-09,2024-01-19T07:50:45.527000+00:00
17670,17688,contact@electricite-loos.fr,RME Loos (FR-FR1),"[3.014273, 50.615449]",12 Rue Salengro LOOS,FRFR1EAYRU,"Loos, Parking Salengro",Parking public,3,Accès libre,24/7,FALSE,FRFR1EDXS2,2022-12-09,2024-01-19T07:50:45.527000+00:00
17671,17689,contact@electricite-loos.fr,RME Loos (FR-FR1),"[3.014273, 50.615449]",12 Rue Salengro LOOS,FRFR1EAYRU,"Loos, Parking Salengro",Parking public,3,Accès libre,24/7,FALSE,FRFR1EDXS3,2022-12-09,2024-01-19T07:50:45.527000+00:00


In [122]:
itinerance_init.loc[itinerance_init.nom_enseigne == 'RME Loos (FR-FR1)', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
17660,17678,contact@electricite-loos.fr,RME Loos (FR-FR1),"[3.02,50.61]",rue du colonnel jean colonna d'ornano loos,FRFR1EAYRU,"Loos, Parking de la Gare",Parking public,3,Accès libre,24/7,false,FRFR1EAYRU1,2022-12-25,2024-01-19T07:49:57.766000+00:00
17661,17679,contact@electricite-loos.fr,RME Loos (FR-FR1),"[3.02,50.61]",rue du colonnel ornano loos,FRFR1EAYRU,"Loos, Parking de la Gare",Parking public,3,Accès libre,24/7,false,FRFR1EAYRU2,2022-08-25,2024-01-19T07:49:57.766000+00:00
17669,17687,contact@electricite-loos.fr,RME Loos (FR-FR1),"[3.014273, 50.615449]",12 Rue Salengro LOOS,FRFR1EAYRU,"Loos, Parking Salengro",Parking public,3,Accès libre,24/7,FALSE,FRFR1EDXS1,2022-12-09,2024-01-19T07:50:45.527000+00:00
17670,17688,contact@electricite-loos.fr,RME Loos (FR-FR1),"[3.014273, 50.615449]",12 Rue Salengro LOOS,FRFR1EAYRU,"Loos, Parking Salengro",Parking public,3,Accès libre,24/7,FALSE,FRFR1EDXS2,2022-12-09,2024-01-19T07:50:45.527000+00:00
17685,17703,contact@electricite-loos.fr,RME Loos (FR-FR1),"[3.009613, 50.617509]",12 Rue Salengro LOOS,FRFR1EPJXS,"Loos, Parking CIL",Parking public,3,Accès libre,24/7,FALSE,FRFR1EPJXS1,2022-10-04,2024-01-19T07:49:31.469000+00:00
17686,17704,contact@electricite-loos.fr,RME Loos (FR-FR1),"[3.009613, 50.617509]",12 Rue Salengro LOOS,FRFR1EPJXS,"Loos, Parking CIL",Parking public,3,Accès libre,24/7,FALSE,FRFR1EPJXS2,2022-10-04,2024-01-19T07:49:31.469000+00:00


### cohérence station - horaires
- exemple de station avec plusieurs types d'horaire (15)

In [123]:
itinerance_init.loc[itinerance_init.id_station_itinerance == 'FRS23D2302001', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
51326,53380,contact@mobive.fr,Réseau de recharge Creuse,"[2.297630254367735, 45.98220366857867]",Place du Treix,FRS23D2302001,BELLEGARDE EN MARCHE - PLACE DE TREIX,Parking public,2,Accès libre,Mo-Su 00:00-00:01,False,FRS23P230200101,2023-03-09,2024-04-30T07:18:04.678000+00:00
51327,53381,contact@mobive.fr,Réseau de recharge Creuse,"[2.297630254367735, 45.98220366857867]",Place du Treix,FRS23D2302001,BELLEGARDE EN MARCHE - PLACE DE TREIX,Parking public,2,Accès libre,Mo-Su 00:00-00:02,False,FRS23P230200102,2023-03-09,2024-04-30T07:18:04.678000+00:00


### cohérence station - deux-roues
- exemple de station avec plusieurs indications d'accessibilité deux-roues (583)

In [124]:
itinerance_init.loc[itinerance_init.id_station_itinerance == 'FRV75PPX1716', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
73289,76084,supervision-ev.france@totalenergies.com,Belib',"[2.3242009, 48.8957574]",47 Rue Navier 75017 Paris,FRV75PPX1716,Paris | Rue Navier 47,Voirie,7,Accès libre,24/7,TRUE,FRV75EPX17161,2023-07-06,2024-04-11T13:40:44.232000+00:00
73290,76085,supervision-ev.france@totalenergies.com,Belib',"[2.3242009, 48.8957574]",47 Rue Navier 75017 Paris,FRV75PPX1716,Paris | Rue Navier 47,Voirie,7,Accès libre,24/7,TRUE,FRV75EPX17162,2023-07-06,2024-04-11T13:40:44.232000+00:00
73291,76086,supervision-ev.france@totalenergies.com,Belib',"[2.3242009, 48.8957574]",47 Rue Navier 75017 Paris,FRV75PPX1716,Paris | Rue Navier 47,Voirie,7,Accès libre,24/7,FALSE,FRV75EPX17163,2023-07-06,2024-04-11T13:40:44.232000+00:00
73292,76087,supervision-ev.france@totalenergies.com,Belib',"[2.3242009, 48.8957574]",47 Rue Navier 75017 Paris,FRV75PPX1716,Paris | Rue Navier 47,Voirie,7,Accès libre,24/7,FALSE,FRV75EPX17164,2023-07-06,2024-04-11T13:40:44.232000+00:00
73293,76088,supervision-ev.france@totalenergies.com,Belib',"[2.3242009, 48.8957574]",47 Rue Navier 75017 Paris,FRV75PPX1716,Paris | Rue Navier 47,Voirie,7,Accès libre,24/7,FALSE,FRV75EPX17165,2023-07-06,2024-04-11T13:40:44.232000+00:00
73294,76089,supervision-ev.france@totalenergies.com,Belib',"[2.3242009, 48.8957574]",47 Rue Navier 75017 Paris,FRV75PPX1716,Paris | Rue Navier 47,Voirie,7,Accès libre,24/7,FALSE,FRV75EPX17166,2023-07-06,2024-04-11T13:40:44.232000+00:00
73295,76090,supervision-ev.france@totalenergies.com,Belib',"[2.3242009, 48.8957574]",47 Rue Navier 75017 Paris,FRV75PPX1716,Paris | Rue Navier 47,Voirie,7,Accès libre,24/7,FALSE,FRV75EPX17167,2023-07-06,2024-04-11T13:40:44.232000+00:00
